In [1]:
from __future__ import print_function
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 2 
epochs = 100  
latent_dim = 128  # the dim of embedded space
num_samples = 10000  # max_training samples
# w2vec data dir
data_path = "data-bin/stocks_emb.w2vec"

# collections of input/output
input_texts = []
target_texts = []
input_symbols = set()
target_symbols = set()

with open(data_path, "r") as f:
    lines = f.read().split('\n')  #embedded vectors reading
f1 = [item.split(' ') for item in lines[1:]]
word2vec = {item[0]: np.array(list(map(float, item[1:]))) for item in f1}
#only 250 sample pairs
list1 = [['PHUN,NBSE,OCGN,BOXL,KTOV,BTAI,WHLR,XFOR,AZRX,SLRX,IMTE,EDAP,ENG,IZEA,SGOC,CHMA,VRML,NVUS,USEG,APOP', 'PHUN,CGIX,SLRX,EPZM,DRRX,TMDI,SUNW,NEOS,YTEN,TBIO,EDUC,OCGN,CTIC,STRO,PULM,EPIX,BHTG,MGEN,IMUX,DLPN'], ['PHUN,CGIX,SLRX,EPZM,DRRX,TMDI,SUNW,NEOS,YTEN,TBIO,EDUC,OCGN,CTIC,STRO,PULM,EPIX,BHTG,MGEN,IMUX,DLPN', 'AXSM,ADXS,INSM,SAGE,PDSB,RBZ,VERO,BCDA,MICT,SNSS,BLPH,BHTG,EQ,SCYX,ROKU,OXBR,CLPS,RETA,TLC,IPWR'], ['AXSM,ADXS,INSM,SAGE,PDSB,RBZ,VERO,BCDA,MICT,SNSS,BLPH,BHTG,EQ,SCYX,ROKU,OXBR,CLPS,RETA,TLC,IPWR', 'CLPS,MBOT,YI,PHUN,PLXP,ORGO,VTVT,AXSM,TCMD,XTLB,NOVN,TMDI,MRIN,IMBI,ICON,SPI,CKPT,VIOT,FUV,TLRY'], ['CLPS,MBOT,YI,PHUN,PLXP,ORGO,VTVT,AXSM,TCMD,XTLB,NOVN,TMDI,MRIN,IMBI,ICON,SPI,CKPT,VIOT,FUV,TLRY', 'ORGO,PHUN,WINS,SAEX,VIVE,ATOS,GLG,SLS,DCAR,AMTBB,KNDI,IPDN,ZKIN,AZRX,HUGE,OPGN,SNDE,RCEL,MTP,HIMX'], ['ORGO,PHUN,WINS,SAEX,VIVE,ATOS,GLG,SLS,DCAR,AMTBB,KNDI,IPDN,ZKIN,AZRX,HUGE,OPGN,SNDE,RCEL,MTP,HIMX', 'PROF,ORGO,BRQS,PRPO,PLXP,DCIX,INPX,AEHR,SAEX,AMRN,NOVN,HEBT,AMRH,SNSS,IMBI,BBBY,BVSN,GSM,RBZ,AQMS'], ['PROF,ORGO,BRQS,PRPO,PLXP,DCIX,INPX,AEHR,SAEX,AMRN,NOVN,HEBT,AMRH,SNSS,IMBI,BBBY,BVSN,GSM,RBZ,AQMS', 'TBLT,MOXC,TBLTW,SAEX,SNSS,ARPO,YRCW,FRAN,OMEX,TLRY,HYRE,BCDA,WTER,YGYI,SEEL,UROV,EPIX,PTSI,AMTX,TWMC'], ['TBLT,MOXC,TBLTW,SAEX,SNSS,ARPO,YRCW,FRAN,OMEX,TLRY,HYRE,BCDA,WTER,YGYI,SEEL,UROV,EPIX,PTSI,AMTX,TWMC', 'MBOT,BIOC,FTFT,VVUS,INVE,CPST,UONE,WTER,NEON,CBAT,HTBX,BOXL,LIQT,REXN,PHUN,NXTD,CODX,DCAR,EDSA,NBEV'], ['MBOT,BIOC,FTFT,VVUS,INVE,CPST,UONE,WTER,NEON,CBAT,HTBX,BOXL,LIQT,REXN,PHUN,NXTD,CODX,DCAR,EDSA,NBEV', 'KTOV,TYME,CCCL,PHUN,RESN,BSQR,CKPT,CHFS,ACHV,QFIN,VRML,CBAT,ELGX,NAKD,NURO,FBIO,AYTU,KZIA,NXTD,EVSI'], ['KTOV,TYME,CCCL,PHUN,RESN,BSQR,CKPT,CHFS,ACHV,QFIN,VRML,CBAT,ELGX,NAKD,NURO,FBIO,AYTU,KZIA,NXTD,EVSI', 'TBLT,TBLTW,BIOC,ALRN,ENTX,VERB,TMDI,IMTE,ADXS,NOVN,VCNX,HJLI,MERC,NWGI,CERC,ALT,CLPS,IPWR,VERO,CPST'], ['TBLT,TBLTW,BIOC,ALRN,ENTX,VERB,TMDI,IMTE,ADXS,NOVN,VCNX,HJLI,MERC,NWGI,CERC,ALT,CLPS,IPWR,VERO,CPST', 'LMFA,PAVMW,HHT,BIOC,CCCL,CORV,SNCA,SAEX,AMTX,LPTX,CGIX,RBZ,LPCN,SPI,TCON,AGEN,LLIT,RWLK,MRIN,NEBUW'], ['LMFA,PAVMW,HHT,BIOC,CCCL,CORV,SNCA,SAEX,AMTX,LPTX,CGIX,RBZ,LPCN,SPI,TCON,AGEN,LLIT,RWLK,MRIN,NEBUW', 'PROF,MOXC,AVCO,QMCO,CBAT,VERO,CLDX,ICON,MARK,OZK,WKHS,FANH,AGRX,CORV,PHUN,SES,IZEA,PLXP,BURG,LLIT'], ['PROF,MOXC,AVCO,QMCO,CBAT,VERO,CLDX,ICON,MARK,OZK,WKHS,FANH,AGRX,CORV,PHUN,SES,IZEA,PLXP,BURG,LLIT', 'AVCO,BRID,BOXL,VERB,CGIX,TBLTW,STAF,NVUS,TVIX,ALT,APEN,TMDI,XTLB,BVSN,ATLC,SES,SNSS,CGEN,AVGR,CARV'], ['AVCO,BRID,BOXL,VERB,CGIX,TBLTW,STAF,NVUS,TVIX,ALT,APEN,TMDI,XTLB,BVSN,ATLC,SES,SNSS,CGEN,AVGR,CARV', 'AMCI,SPI,BOXL,BLNK,IFMK,MBOT,RETO,RCON,RIOT,CIDM,ALPN,XTLB,ATHX,AOSL,EARS,EHTH,LFVN,VERO,FRAN,ARAY'], ['AMCI,SPI,BOXL,BLNK,IFMK,MBOT,RETO,RCON,RIOT,CIDM,ALPN,XTLB,ATHX,AOSL,EARS,EHTH,LFVN,VERO,FRAN,ARAY', 'TBLTW,KZIA,ASLN,OTLK,BKYI,CRIS,RWLK,TBLT,ONTX,APDN,XLNX,ACHV,LRCX,ALPN,USAP,ATOM,INSE,SLM,AZPN,CLRO'], ['TBLTW,KZIA,ASLN,OTLK,BKYI,CRIS,RWLK,TBLT,ONTX,APDN,XLNX,ACHV,LRCX,ALPN,USAP,ATOM,INSE,SLM,AZPN,CLRO', 'PROF,RAND,HYRE,CHEK,TSRI,DGLY,ADXS,MRSN,SRRK,PHUN,OSTK,AGYS,CLRO,TOCA,TLSA,CLMT,OSIS,ICLK,CLDX,ABUS'], ['PROF,RAND,HYRE,CHEK,TSRI,DGLY,ADXS,MRSN,SRRK,PHUN,OSTK,AGYS,CLRO,TOCA,TLSA,CLMT,OSIS,ICLK,CLDX,ABUS', 'TRVN,AVCO,WHLR,FBIO,DTEA,TRNX,ANTE,PHUN,REFR,YGYI,RBZ,CRON,GRIN,GMDA,AUDC,AGMH,UNAM,ANIX,CYCCP,HNNA'], ['TRVN,AVCO,WHLR,FBIO,DTEA,TRNX,ANTE,PHUN,REFR,YGYI,RBZ,CRON,GRIN,GMDA,AUDC,AGMH,UNAM,ANIX,CYCCP,HNNA', 'VYGR,HJLIW,ROSEU,TROV,ZYNE,CNAT,JVA,AVDL,ONCS,SANM,KZIA,CODX,NVCN,WWD,NDRA,CSBR,JJSF,DGLY,PDEX,SOLO'], ['VYGR,HJLIW,ROSEU,TROV,ZYNE,CNAT,JVA,AVDL,ONCS,SANM,KZIA,CODX,NVCN,WWD,NDRA,CSBR,JJSF,DGLY,PDEX,SOLO', 'CODX,WORX,CREG,PHUN,EYEN,TMDI,BKYI,BVSN,CLDX,AMD,ONTX,NVUS,MRCY,CTIC,BIOL,NTRP,ADIL,HBIO,CAPR,MCBS'], ['CODX,WORX,CREG,PHUN,EYEN,TMDI,BKYI,BVSN,CLDX,AMD,ONTX,NVUS,MRCY,CTIC,BIOL,NTRP,ADIL,HBIO,CAPR,MCBS', 'TMSR,PHUN,OCGN,SCYX,NVUS,FBIO,ROSEU,MGTX,ONTX,PETZ,CLDX,USIO,ONCT,FLWS,CFMS,CHCI,ARTL,DOGZ,EGOV,GLIBA'], ['TMSR,PHUN,OCGN,SCYX,NVUS,FBIO,ROSEU,MGTX,ONTX,PETZ,CLDX,USIO,ONCT,FLWS,CFMS,CHCI,ARTL,DOGZ,EGOV,GLIBA', 'TMSR,TBLTW,IMTE,TTNP,ALRN,GRVY,ADIL,IPDN,WINS,MICT,CETXP,FBIO,INFN,PULM,TUES,CLRO,CLWT,DGII,CLDX,DGLY'], ['TMSR,TBLTW,IMTE,TTNP,ALRN,GRVY,ADIL,IPDN,WINS,MICT,CETXP,FBIO,INFN,PULM,TUES,CLRO,CLWT,DGII,CLDX,DGLY', 'MARA,DXYN,QUIK,ALRN,NSSC,APDN,CLDX,JVA,EYEN,TRPX,CNST,AMTX,GERN,UEPS,PDSB,WTER,BLRX,ARVN,AVDL,MOXC'], ['MARA,DXYN,QUIK,ALRN,NSSC,APDN,CLDX,JVA,EYEN,TRPX,CNST,AMTX,GERN,UEPS,PDSB,WTER,BLRX,ARVN,AVDL,MOXC', 'PROF,WORX,AMRS,EOLS,CETXP,VERB,TRVN,IMTE,RKDA,DUOT,FRAN,USIO,UONE,BCOM,LBTYB,XBIT,MICT,SPI,TMSR,ALRN'], ['PROF,WORX,AMRS,EOLS,CETXP,VERB,TRVN,IMTE,RKDA,DUOT,FRAN,USIO,UONE,BCOM,LBTYB,XBIT,MICT,SPI,TMSR,ALRN', 'MGNX,APDN,CETX,NWGI,ADXS,NTGN,PTVCB,BTAI,XBIT,AXGT,DTSS,MBIO,PTVCA,TZOO,ZLAB,SNES,IMGN,EYEN,CODA,CLPS'], ['MGNX,APDN,CETX,NWGI,ADXS,NTGN,PTVCB,BTAI,XBIT,AXGT,DTSS,MBIO,PTVCA,TZOO,ZLAB,SNES,IMGN,EYEN,CODA,CLPS', 'PAYS,NBSE,APOP,CLSK,EOLS,CORV,NRBO,FWP,CLPS,ATOS,CSGS,DLPN,VCNX,CLDX,OSS,ENSG,NVUS,ZFGN,VBIV,TLC'], ['PAYS,NBSE,APOP,CLSK,EOLS,CORV,NRBO,FWP,CLPS,ATOS,CSGS,DLPN,VCNX,CLDX,OSS,ENSG,NVUS,ZFGN,VBIV,TLC', 'EGAN,MARA,EYEN,MBIO,NK,RIOT,MAT,VERB,UEPS,SELB,CLSK,OMCL,FSCT,ARWR,CNXN,EA,COLM,CUE,BTAI,AVXL'], ['EGAN,MARA,EYEN,MBIO,NK,RIOT,MAT,VERB,UEPS,SELB,CLSK,OMCL,FSCT,ARWR,CNXN,EA,COLM,CUE,BTAI,AVXL', 'CLPS,SPI,SSNT,VERB,CREX,NMRD,PTI,ZYXI,CPAH,WORX,REPL,ENOB,ATEC,SCYX,ENTX,GFN,NWGI,NAKD,MGTA,MRKR'], ['CLPS,SPI,SSNT,VERB,CREX,NMRD,PTI,ZYXI,CPAH,WORX,REPL,ENOB,ATEC,SCYX,ENTX,GFN,NWGI,NAKD,MGTA,MRKR', 'PROF,APDN,BKYI,GSUM,ALRN,SIC,ADMA,WORX,CLSK,QTT,BNTC,ISIG,MDRR,REFR,MARA,BHF,MIME,ATEC,IPDN,HBP'], ['PROF,APDN,BKYI,GSUM,ALRN,SIC,ADMA,WORX,CLSK,QTT,BNTC,ISIG,MDRR,REFR,MARA,BHF,MIME,ATEC,IPDN,HBP', 'SOLO,CLSK,LSCC,BIOL,RWLK,PIXY,GRNQ,HBP,ICON,MOSY,SINT,UXIN,CLIR,IFMK,CNST,CIDM,BLNK,GRVY,RKDA,CUE'], ['SOLO,CLSK,LSCC,BIOL,RWLK,PIXY,GRNQ,HBP,ICON,MOSY,SINT,UXIN,CLIR,IFMK,CNST,CIDM,BLNK,GRVY,RKDA,CUE', 'FUV,IFMK,PULM,RBZ,LMNL,OTLK,CAAS,DUOT,GSUM,OXBR,CYBR,GWGH,PRPO,PLUG,EFOI,IMUX,VERB,RTTR,LQDA,UROV'], ['FUV,IFMK,PULM,RBZ,LMNL,OTLK,CAAS,DUOT,GSUM,OXBR,CYBR,GWGH,PRPO,PLUG,EFOI,IMUX,VERB,RTTR,LQDA,UROV', 'PROF,SKYS,IPWR,OCC,PHCF,TBIO,CFMS,SMCI,TLND,SNCA,TEUM,XTLB,ALJJ,CPSI,SAEX,CLDX,AIRG,BSGM,RNWK,CERC'], ['PROF,SKYS,IPWR,OCC,PHCF,TBIO,CFMS,SMCI,TLND,SNCA,TEUM,XTLB,ALJJ,CPSI,SAEX,CLDX,AIRG,BSGM,RNWK,CERC', 'ARTW,MARA,RIOT,IDEX,ENLV,MARPS,FCEL,INVE,CHFS,XNET,MRAM,SKYS,PESI,MNOV,RTTR,TPCO,FTFT,GROW,HJLIW,NXTD'], ['ARTW,MARA,RIOT,IDEX,ENLV,MARPS,FCEL,INVE,CHFS,XNET,MRAM,SKYS,PESI,MNOV,RTTR,TPCO,FTFT,GROW,HJLIW,NXTD', 'KNDI,ABIO,IDEX,AKER,BLNK,LMAT,CLSD,MRAM,AZRX,DTSS,AXNX,FCEL,GRMN,EYEN,ACMR,CLSK,OSPN,LNTH,GALT,MYT'], ['KNDI,ABIO,IDEX,AKER,BLNK,LMAT,CLSD,MRAM,AZRX,DTSS,AXNX,FCEL,GRMN,EYEN,ACMR,CLSK,OSPN,LNTH,GALT,MYT', 'ZSAN,ONCT,INFI,LOPE,NAOV,ADIL,CAR,OMEX,CLSK,HTGM,ENG,IDRA,DYAI,ACHV,SSSS,ECOR,KZIA,TRCH,ACIU,TRIB'], ['ZSAN,ONCT,INFI,LOPE,NAOV,ADIL,CAR,OMEX,CLSK,HTGM,ENG,IDRA,DYAI,ACHV,SSSS,ECOR,KZIA,TRCH,ACIU,TRIB', 'PROF,ACHV,CIFS,BOOM,TTD,Z,ROKU,ZG,OLED,VYGR,LQDA,IQ,APDN,CATM,XOG,MRNS,ACIA,MARA,ECOR,AQMS'], ['PROF,ACHV,CIFS,BOOM,TTD,Z,ROKU,ZG,OLED,VYGR,LQDA,IQ,APDN,CATM,XOG,MRNS,ACIA,MARA,ECOR,AQMS', 'CIFS,CREG,QURE,TTNP,CRSP,IMTE,VERB,GNCA,AVRO,VERI,GLG,STMP,SPI,AKCA,HX,AUPH,ADVM,RGNX,PULM,JRJC'], ['CIFS,CREG,QURE,TTNP,CRSP,IMTE,VERB,GNCA,AVRO,VERI,GLG,STMP,SPI,AKCA,HX,AUPH,ADVM,RGNX,PULM,JRJC', 'PROF,MTP,GNCA,SMMT,IFMK,HSII,GSM,SESN,PIH,MARPS,HLIO,IMUX,REED,DCAR,BLNK,HTBX,BIOL,NTGN,EVER,MNKD'], ['PROF,MTP,GNCA,SMMT,IFMK,HSII,GSM,SESN,PIH,MARPS,HLIO,IMUX,REED,DCAR,BLNK,HTBX,BIOL,NTGN,EVER,MNKD', 'MTP,BVSN,TNDM,SMMT,MELI,AVGR,HOVNP,FTR,XNET,RGLS,MGTX,DFFN,IONS,LMNL,VERO,SUPN,GWPH,CLVS,MGRC,OSTK'], ['MTP,BVSN,TNDM,SMMT,MELI,AVGR,HOVNP,FTR,XNET,RGLS,MGTX,DFFN,IONS,LMNL,VERO,SUPN,GWPH,CLVS,MGRC,OSTK', 'RKDA,SQBG,HZNP,TGTX,BPTH,CSBR,TWNKW,COLL,GH,CIFS,NVCN,CETXP,TROV,TTOO,IFMK,CRAI,ARTL,CJJD,CPAH,HAIN'], ['RKDA,SQBG,HZNP,TGTX,BPTH,CSBR,TWNKW,COLL,GH,CIFS,NVCN,CETXP,TROV,TTOO,IFMK,CRAI,ARTL,CJJD,CPAH,HAIN', 'PROF,BPTH,NH,CIFS,PBYI,LXRX,ZS,AQMS,APLS,SQBG,WHLRP,XBIT,XRAY,NK,DGLY,SRCL,ACHC,FOLD,CKPT,FTEK'], ['PROF,BPTH,NH,CIFS,PBYI,LXRX,ZS,AQMS,APLS,SQBG,WHLRP,XBIT,XRAY,NK,DGLY,SRCL,ACHC,FOLD,CKPT,FTEK', 'ASND,ICON,JRJC,CIFS,PETZ,RLMD,NCTY,ATEC,CHCI,MOXC,HJLIW,PHUN,TYHT,TCON,AGRX,BNTC,PTE,LUNA,DGLY,IFMK'], ['ASND,ICON,JRJC,CIFS,PETZ,RLMD,NCTY,ATEC,CHCI,MOXC,HJLIW,PHUN,TYHT,TCON,AGRX,BNTC,PTE,LUNA,DGLY,IFMK', 'PROF,PETZ,RTTR,HEBT,TYHT,TOUR,TCOM,AQMS,MOXC,BNGO,BHTG,MTP,LYL,NVCN,NCTY,QTT,FORK,ANTE,AVXL,ARTL'], ['PROF,PETZ,RTTR,HEBT,TYHT,TOUR,TCOM,AQMS,MOXC,BNGO,BHTG,MTP,LYL,NVCN,NCTY,QTT,FORK,ANTE,AVXL,ARTL', 'BPTH,NBSE,TENX,AYTU,TARA,SNES,DFFN,VRML,LMFA,HGSH,PHUN,ICON,LLIT,RLMD,DOGZ,TSG,ALT,RBZ,TLSA,IPDN'], ['BPTH,NBSE,TENX,AYTU,TARA,SNES,DFFN,VRML,LMFA,HGSH,PHUN,ICON,LLIT,RLMD,DOGZ,TSG,ALT,RBZ,TLSA,IPDN', 'BPTH,SEEL,ONCT,RBZ,CBLI,DFFN,ALT,GEC,ENLV,SNSS,TROV,CLMT,ESEA,AKER,NH,INPX,JNCE,AMRH,TWMC,TNXP'], ['BPTH,SEEL,ONCT,RBZ,CBLI,DFFN,ALT,GEC,ENLV,SNSS,TROV,CLMT,ESEA,AKER,NH,INPX,JNCE,AMRH,TWMC,TNXP', 'PROF,AQB,PULM,GLBS,DXPE,TNXP,EDSA,BLIN,TRNX,NATR,TENX,RARX,LPTX,UPLD,OXBR,PEIX,CNST,CLSK,TROV,AUTO'], ['PROF,AQB,PULM,GLBS,DXPE,TNXP,EDSA,BLIN,TRNX,NATR,TENX,RARX,LPTX,UPLD,OXBR,PEIX,CNST,CLSK,TROV,AUTO', 'GLBS,BPTH,PXS,ORGO,YTRA,ICON,DCAR,JAGX,VRML,AGRX,PRTH,MCRB,NAKD,MDRR,PTE,JRJC,SKYS,TNXP,BIOC,SNOA'], ['GLBS,BPTH,PXS,ORGO,YTRA,ICON,DCAR,JAGX,VRML,AGRX,PRTH,MCRB,NAKD,MDRR,PTE,JRJC,SKYS,TNXP,BIOC,SNOA', 'VVPR,ALRN,NDRA,ZN,HYRE,SAEX,SFIX,ADXS,SEEL,ANY,JNCE,ASLN,MRSN,STML,IPWR,ONCT,PHUN,AGRX,JAN,AIHS'], ['VVPR,ALRN,NDRA,ZN,HYRE,SAEX,SFIX,ADXS,SEEL,ANY,JNCE,ASLN,MRSN,STML,IPWR,ONCT,PHUN,AGRX,JAN,AIHS', 'AKTX,NWGI,CLNE,GH,LEDS,CNAT,DFFN,ADXS,TEUM,VRA,SES,RAVE,NTRA,PHUN,HJLI,NBRV,TWMC,CRIS,NCNA,XCUR'], ['AKTX,NWGI,CLNE,GH,LEDS,CNAT,DFFN,ADXS,TEUM,VRA,SES,RAVE,NTRA,PHUN,HJLI,NBRV,TWMC,CRIS,NCNA,XCUR', 'ATOS,OXBR,RTTR,GNPX,BIOC,BBCP,SMSI,AIHS,HUGE,MDB,SCYX,INFI,DCAR,TNXP,SNSS,TWMC,DOMO,YTEN,HJLI,PULM'], ['ATOS,OXBR,RTTR,GNPX,BIOC,BBCP,SMSI,AIHS,HUGE,MDB,SCYX,INFI,DCAR,TNXP,SNSS,TWMC,DOMO,YTEN,HJLI,PULM', 'PROF,SLNO,CELH,TENX,XFOR,AVID,MGTA,CATS,ACHV,AVGR,SRTS,LPTX,DRAD,CWBR,CFMS,BNGO,FRAN,FAMI,CATB,FEIM'], ['PROF,SLNO,CELH,TENX,XFOR,AVID,MGTA,CATS,ACHV,AVGR,SRTS,LPTX,DRAD,CWBR,CFMS,BNGO,FRAN,FAMI,CATB,FEIM', 'JAGX,DARE,PHAS,SELB,GRTS,ATOS,SLRX,ICON,ACRS,RMTI,MOSY,MESO,SRNE,AVGR,VIOT,YVR,CLRB,SND,DZSI,INNT'], ['JAGX,DARE,PHAS,SELB,GRTS,ATOS,SLRX,ICON,ACRS,RMTI,MOSY,MESO,SRNE,AVGR,VIOT,YVR,CLRB,SND,DZSI,INNT', 'TYHT,VXRT,SRNE,DMPI,TRMD,NCTY,OCGN,ITRM,DRAD,BCOR,AYTU,STNE,ARAV,MGEN,NDRA,STAF,IDEX,ZYNE,ALT,MYOS'], ['TYHT,VXRT,SRNE,DMPI,TRMD,NCTY,OCGN,ITRM,DRAD,BCOR,AYTU,STNE,ARAV,MGEN,NDRA,STAF,IDEX,ZYNE,ALT,MYOS', 'CFRX,SFET,SOLO,CODA,CFMS,CYTK,RBZ,WHLR,NVAX,CYCC,GLG,CPRX,AVCO,HEBT,YVR,BLCM,CYRX,SNCA,LYL,TTNP'], ['CFRX,SFET,SOLO,CODA,CFMS,CYTK,RBZ,WHLR,NVAX,CYCC,GLG,CPRX,AVCO,HEBT,YVR,BLCM,CYRX,SNCA,LYL,TTNP', 'SPI,PHUN,SRNE,DCIX,RCEL,OTLK,ATHE,LMB,TLSA,VERB,FTEK,SOLO,SLNO,LIFE,VCNX,SCON,STNE,NDRA,HOVNP,VKTX'], ['SPI,PHUN,SRNE,DCIX,RCEL,OTLK,ATHE,LMB,TLSA,VERB,FTEK,SOLO,SLNO,LIFE,VCNX,SCON,STNE,NDRA,HOVNP,VKTX', 'PROF,SNSS,DRIO,TVIX,HIBB,BPTH,DMAC,SEEL,ARDS,IMMP,IDN,MYT,RCEL,SRNE,DISCB,VIIX,HOVNP,CPIX,MDCA,CPAH'], ['PROF,SNSS,DRIO,TVIX,HIBB,BPTH,DMAC,SEEL,ARDS,IMMP,IDN,MYT,RCEL,SRNE,DISCB,VIIX,HOVNP,CPIX,MDCA,CPAH', 'NK,CRIS,CPRX,AXSM,NWGI,CHEK,CHCI,CTXR,IDEX,FTEK,CCXI,CFMS,AVXL,SNES,ASFI,OPCH,ITRM,REPL,PAYS,TSRI'], ['NK,CRIS,CPRX,AXSM,NWGI,CHEK,CHCI,CTXR,IDEX,FTEK,CCXI,CFMS,AVXL,SNES,ASFI,OPCH,ITRM,REPL,PAYS,TSRI', 'CPAH,DCAR,ALDX,PHAS,SHIP,LTRX,OXBR,BBBY,BIOC,SPI,DUOT,ORTX,WINS,TYHT,MOXC,IDN,EDRY,RBZ,HYRE,IMBI'], ['CPAH,DCAR,ALDX,PHAS,SHIP,LTRX,OXBR,BBBY,BIOC,SPI,DUOT,ORTX,WINS,TYHT,MOXC,IDN,EDRY,RBZ,HYRE,IMBI', 'PROF,RBZ,GRNQ,AVEO,SRAX,NMCI,CARV,IMTE,UONE,AMRS,OPHC,SCVL,MDCA,IMBI,ARCT,TMSR,SNDX,JAGX,CHFS,MTP'], ['PROF,RBZ,GRNQ,AVEO,SRAX,NMCI,CARV,IMTE,UONE,AMRS,OPHC,SCVL,MDCA,IMBI,ARCT,TMSR,SNDX,JAGX,CHFS,MTP', 'CLWT,TCDA,CIFS,SFET,HHT,ELTK,HOVNP,TENX,PETZ,SNSS,YVR,FFHL,MYOS,VRNT,LULU,ENLV,CSSE,TAIT,KELYB,HIHO'], ['CLWT,TCDA,CIFS,SFET,HHT,ELTK,HOVNP,TENX,PETZ,SNSS,YVR,FFHL,MYOS,VRNT,LULU,ENLV,CSSE,TAIT,KELYB,HIHO', 'PROF,CSSE,APDN,ICON,GLPG,AVEO,SEEL,VKTX,YMAB,MRUS,PRGS,ADMA,ROSEU,HJLIW,YI,TYHT,SGLB,HX,AVNW,AEZS'], ['PROF,CSSE,APDN,ICON,GLPG,AVEO,SEEL,VKTX,YMAB,MRUS,PRGS,ADMA,ROSEU,HJLIW,YI,TYHT,SGLB,HX,AVNW,AEZS', 'PULM,KXIN,AVEO,PHAS,JAN,TROV,TBIO,VCNX,CAPR,LTBR,ATOM,PTNR,VERB,IPWR,CATS,SORL,EVOK,THTX,EYEG,ALT'], ['PULM,KXIN,AVEO,PHAS,JAN,TROV,TBIO,VCNX,CAPR,LTBR,ATOM,PTNR,VERB,IPWR,CATS,SORL,EVOK,THTX,EYEG,ALT', 'AVEO,EDAP,SGMO,ADMA,RIOT,TCCO,OXBR,NXTD,ZYNE,TARA,MARA,SUNW,CNAT,TCON,DARE,MNOV,GURE,LTBR,BHTG,TWMC'], ['AVEO,EDAP,SGMO,ADMA,RIOT,TCCO,OXBR,NXTD,ZYNE,TARA,MARA,SUNW,CNAT,TCON,DARE,MNOV,GURE,LTBR,BHTG,TWMC', 'OPTT,EVGN,OESX,IOVA,VERI,TLSA,FTEK,CLSN,AMRS,SRAX,BOOM,AQB,SRNE,ADAP,ANY,SNSS,ZYNE,QRHC,RIOT,MARA'], ['OPTT,EVGN,OESX,IOVA,VERI,TLSA,FTEK,CLSN,AMRS,SRAX,BOOM,AQB,SRNE,ADAP,ANY,SNSS,ZYNE,QRHC,RIOT,MARA', 'TYHT,REPH,PNRG,CFRX,MTP,TNXP,SGBX,AMRS,XOG,OTLK,CIFS,TLSA,ASYS,ICAD,SNSS,DBVT,LQDA,FORTY,ANTE,PXS'], ['TYHT,REPH,PNRG,CFRX,MTP,TNXP,SGBX,AMRS,XOG,OTLK,CIFS,TLSA,ASYS,ICAD,SNSS,DBVT,LQDA,FORTY,ANTE,PXS', 'PROF,IMUX,RMBL,AEHR,TWMC,DNJR,PRPO,AVEO,ORMP,PSTV,KNSA,IFMK,FVE,HJLI,TCCO,ZYNE,SLRX,REPL,MRNA,IMTE'], ['PROF,IMUX,RMBL,AEHR,TWMC,DNJR,PRPO,AVEO,ORMP,PSTV,KNSA,IFMK,FVE,HJLI,TCCO,ZYNE,SLRX,REPL,MRNA,IMTE', 'PRPO,OCGN,RCEL,CETXP,IMUX,NBEV,NIU,PSTV,WWR,PHAS,EFOI,RELV,DCAR,QUIK,CLDX,DNJR,LEDS,FTEK,TTOO,MOSY'], ['PRPO,OCGN,RCEL,CETXP,IMUX,NBEV,NIU,PSTV,WWR,PHAS,EFOI,RELV,DCAR,QUIK,CLDX,DNJR,LEDS,FTEK,TTOO,MOSY', 'HQI,SUNW,PRPO,CMRX,EFOI,OCGN,AMRS,FARO,CGIX,ROSE,PHAS,PRVB,PLL,CFRX,VRML,ONTX,RLMD,HTGM,SRAX,MOTS'], ['HQI,SUNW,PRPO,CMRX,EFOI,OCGN,AMRS,FARO,CGIX,ROSE,PHAS,PRVB,PLL,CFRX,VRML,ONTX,RLMD,HTGM,SRAX,MOTS', 'INPX,MARA,EYES,TMDI,FLGT,CFRX,PETZ,GROW,SLP,BOXL,VISL,AFH,ICON,GH,AZRX,SES,ARPO,CODA,CATB,RIOT'], ['INPX,MARA,EYES,TMDI,FLGT,CFRX,PETZ,GROW,SLP,BOXL,VISL,AFH,ICON,GH,AZRX,SES,ARPO,CODA,CATB,RIOT', 'FTFT,PHUN,IMUX,IMRN,DTSS,BLIN,TRCH,ENG,NWGI,EVFM,JG,YVR,GRNQ,ZKIN,OSN,HJLIW,WTER,GURE,NSSC,QRHC'], ['FTFT,PHUN,IMUX,IMRN,DTSS,BLIN,TRCH,ENG,NWGI,EVFM,JG,YVR,GRNQ,ZKIN,OSN,HJLIW,WTER,GURE,NSSC,QRHC', 'PROF,TRNX,OCGN,RBZ,SPI,SLRX,ENLV,WISA,VIVE,KTOV,IMRN,VRML,PIXY,DUOT,SNCA,DISCB,ROSE,PHAS,MRUS,LMFA'], ['PROF,TRNX,OCGN,RBZ,SPI,SLRX,ENLV,WISA,VIVE,KTOV,IMRN,VRML,PIXY,DUOT,SNCA,DISCB,ROSE,PHAS,MRUS,LMFA', 'PRPO,BPTH,AMRS,MOGO,TTNP,NMCI,CFMS,HSDT,SLRX,SNES,BOXL,MRUS,ADVM,PHUN,ROSEU,MKGI,ECOR,XFOR,UONE,ASUR'], ['PRPO,BPTH,AMRS,MOGO,TTNP,NMCI,CFMS,HSDT,SLRX,SNES,BOXL,MRUS,ADVM,PHUN,ROSEU,MKGI,ECOR,XFOR,UONE,ASUR', 'QFIN,ICON,JAGX,REXN,QCOM,LOGC,PRPO,SESN,IMV,ASUR,LMB,MVIS,SCPH,AMRS,HSDT,YVR,EPIX,VIOT,RWLK,MICT'], ['QFIN,ICON,JAGX,REXN,QCOM,LOGC,PRPO,SESN,IMV,ASUR,LMB,MVIS,SCPH,AMRS,HSDT,YVR,EPIX,VIOT,RWLK,MICT', 'IPWR,UXIN,NVCN,BOXL,CETXP,PAYS,RBZ,DUOT,XSPA,ZBIO,QCOM,AVEO,PLL,SANW,LPCN,HSDT,RLMD,VNET,BBGI,GOGO'], ['IPWR,UXIN,NVCN,BOXL,CETXP,PAYS,RBZ,DUOT,XSPA,ZBIO,QCOM,AVEO,PLL,SANW,LPCN,HSDT,RLMD,VNET,BBGI,GOGO', 'MBIO,HJLIW,CKPT,NOVN,AMRS,CNAT,QFIN,PRPO,VIOT,UXIN,REPL,MBRX,ADTN,IMUX,REFR,JAN,ECOR,FBIO,MARA,ARVN'], ['MBIO,HJLIW,CKPT,NOVN,AMRS,CNAT,QFIN,PRPO,VIOT,UXIN,REPL,MBRX,ADTN,IMUX,REFR,JAN,ECOR,FBIO,MARA,ARVN', 'PROF,MBRX,LMNL,ICON,TBLTW,HUGE,EYPT,ZYNE,SNCA,FBIO,ROSE,RETO,LTRX,CRNT,JAN,CTG,CYAD,ATOS,MBIO,CKPT'], ['PROF,MBRX,LMNL,ICON,TBLTW,HUGE,EYPT,ZYNE,SNCA,FBIO,ROSE,RETO,LTRX,CRNT,JAN,CTG,CYAD,ATOS,MBIO,CKPT', 'SES,ATOS,ROSEU,XFOR,IMUX,TROV,DMPI,ATHX,AKTX,WKHS,NEOS,WHLR,HAS,BVXV,OPCH,VCNX,ASLN,EIDX,FCEL,TAIT'], ['SES,ATOS,ROSEU,XFOR,IMUX,TROV,DMPI,ATHX,AKTX,WKHS,NEOS,WHLR,HAS,BVXV,OPCH,VCNX,ASLN,EIDX,FCEL,TAIT', 'ENT,HJLIW,AVGR,VICR,SLAB,FTFT,SYNL,MGI,JAN,BIMI,AFH,MTP,SNSS,JAKK,TLSA,FCEL,OBLN,WHLR,TYHT,MVIS'], ['ENT,HJLIW,AVGR,VICR,SLAB,FTFT,SYNL,MGI,JAN,BIMI,AFH,MTP,SNSS,JAKK,TLSA,FCEL,OBLN,WHLR,TYHT,MVIS', 'TLSA,TZOO,SCPH,SCKT,DFFN,AXTI,RBZ,LYL,LQDA,NEBUW,UFPI,INNT,MARK,CPRX,JAN,LIFE,BCOV,ADAP,PFMT,ESTA'], ['TLSA,TZOO,SCPH,SCKT,DFFN,AXTI,RBZ,LYL,LQDA,NEBUW,UFPI,INNT,MARK,CPRX,JAN,LIFE,BCOV,ADAP,PFMT,ESTA', 'PROF,HELE,RGEN,WWR,WKHS,TBBK,MOSY,CASH,TBLTW,TOCA,IMV,ALIM,SURF,HLG,NEPH,EQ,ORRF,LMNL,BNTC,HBP'], ['PROF,HELE,RGEN,WWR,WKHS,TBBK,MOSY,CASH,TBLTW,TOCA,IMV,ALIM,SURF,HLG,NEPH,EQ,ORRF,LMNL,BNTC,HBP', 'TWMC,ATHE,CYOU,SOHU,ENLV,SUNW,ATEC,TAIT,DOGZ,HEBT,NCNA,SESN,VERB,BLIN,SYNL,IDRA,ARAV,OCGN,VTVT,CETX'], ['TWMC,ATHE,CYOU,SOHU,ENLV,SUNW,ATEC,TAIT,DOGZ,HEBT,NCNA,SESN,VERB,BLIN,SYNL,IDRA,ARAV,OCGN,VTVT,CETX', 'NOVN,PI,OXBR,LOVE,PRPO,FCEL,STKS,MKGI,ATEC,BRKS,DWSN,GTHX,SANM,GAIA,RAVE,CGIX,REFR,TAIT,SUNW,BKYI'], ['NOVN,PI,OXBR,LOVE,PRPO,FCEL,STKS,MKGI,ATEC,BRKS,DWSN,GTHX,SANM,GAIA,RAVE,CGIX,REFR,TAIT,SUNW,BKYI', 'PROF,ABIO,PRPO,ENPH,VIVE,TAIT,CFRX,SUNW,PCRX,QUMU,RAVE,OSIS,CWBR,RWLK,AVID,PIXY,AMTBB,DMRC,APYX,STRA'], ['PROF,ABIO,PRPO,ENPH,VIVE,TAIT,CFRX,SUNW,PCRX,QUMU,RAVE,OSIS,CWBR,RWLK,AVID,PIXY,AMTBB,DMRC,APYX,STRA', 'DFFN,DMRC,HDSN,ABIO,MHLD,EARS,HJLIW,MOXC,LWAY,CTXR,JBSS,QTNT,CLCT,ORGO,CYCCP,JAN,VTVT,MRNS,ZYXI,IDEX'], ['DFFN,DMRC,HDSN,ABIO,MHLD,EARS,HJLIW,MOXC,LWAY,CTXR,JBSS,QTNT,CLCT,ORGO,CYCCP,JAN,VTVT,MRNS,ZYXI,IDEX', 'IMBI,DRRX,ZIXI,MERC,CWBR,GEOS,ATOM,MELI,ENT,TAIT,CIFS,IMV,KXIN,LMNL,VRAY,IDRA,CLSK,ANIK,GWGH,UCTT'], ['IMBI,DRRX,ZIXI,MERC,CWBR,GEOS,ATOM,MELI,ENT,TAIT,CIFS,IMV,KXIN,LMNL,VRAY,IDRA,CLSK,ANIK,GWGH,UCTT', 'AMTX,SBGI,ATNX,RAVE,SUNW,OVID,RGLS,BLIN,RTTR,ATOM,PRTH,VIVE,IMGN,DMRC,CLSK,DISCB,OBLN,CKPT,AVGR,TAIT'], ['AMTX,SBGI,ATNX,RAVE,SUNW,OVID,RGLS,BLIN,RTTR,ATOM,PRTH,VIVE,IMGN,DMRC,CLSK,DISCB,OBLN,CKPT,AVGR,TAIT', 'TVIX,COHU,SEDG,ITRI,AKTS,GRWG,WISA,AKRX,BLIN,VIIX,CBAT,RTTR,NWGI,EVER,OBLN,EQ,CENTA,VECO,NEPH,APM'], ['TVIX,COHU,SEDG,ITRI,AKTS,GRWG,WISA,AKRX,BLIN,VIIX,CBAT,RTTR,NWGI,EVER,OBLN,EQ,CENTA,VECO,NEPH,APM', 'WKHS,AKRX,BURG,EVFM,MTSI,ARCE,NH,IOTS,CETXP,ABIO,SGMS,XSPA,MDCA,TAIT,DRAD,ATRO,CHUY,CFMS,POWL,AVRO'], ['WKHS,AKRX,BURG,EVFM,MTSI,ARCE,NH,IOTS,CETXP,ABIO,SGMS,XSPA,MDCA,TAIT,DRAD,ATRO,CHUY,CFMS,POWL,AVRO', 'SPFI,TBLTW,TBLT,ROKU,TWNKW,TTGT,PT,AXNX,FLXN,HEBT,DVAX,ISSC,CECE,ECPG,CODX,OPRX,VERI,NVEE,TSG,MTRX'], ['SPFI,TBLTW,TBLT,ROKU,TWNKW,TTGT,PT,AXNX,FLXN,HEBT,DVAX,ISSC,CECE,ECPG,CODX,OPRX,VERI,NVEE,TSG,MTRX', 'PROF,GENE,CTHR,WPRT,CDLX,NTRA,SRAX,GTHX,GH,CREX,EVFM,CPRX,UBX,SNCR,DYAI,RDVT,ECPG,FSTR,KXIN,ETON'], ['PROF,GENE,CTHR,WPRT,CDLX,NTRA,SRAX,GTHX,GH,CREX,EVFM,CPRX,UBX,SNCR,DYAI,RDVT,ECPG,FSTR,KXIN,ETON', 'TVIX,CLIR,PTSI,CTHR,MARA,GRVY,OPTT,WKHS,VIIX,VISL,CYCCP,MTSL,GROW,JAKK,FTFT,MAYS,SQQQ,WORX,RIOT,ZBIO'], ['TVIX,CLIR,PTSI,CTHR,MARA,GRVY,OPTT,WKHS,VIIX,VISL,CYCCP,MTSL,GROW,JAKK,FTFT,MAYS,SQQQ,WORX,RIOT,ZBIO', 'NSEC,HJLIW,ASFI,NVAX,BPTH,BGFV,MGTX,AEMD,VSTM,SAEX,VTGN,EKSO,ATHX,SPRT,TESS,ARNA,GRVY,LAWS,GPRO,CMRX'], ['NSEC,HJLIW,ASFI,NVAX,BPTH,BGFV,MGTX,AEMD,VSTM,SAEX,VTGN,EKSO,ATHX,SPRT,TESS,ARNA,GRVY,LAWS,GPRO,CMRX', 'TBLTW,ENT,PDSB,ELTK,CLRB,JAKK,VERI,HEBT,LWAY,VIRC,REFR,OBCI,CPRX,DOX,ACER,SNGX,AEIS,DISCB,GENE,LAZY'], ['TBLTW,ENT,PDSB,ELTK,CLRB,JAKK,VERI,HEBT,LWAY,VIRC,REFR,OBCI,CPRX,DOX,ACER,SNGX,AEIS,DISCB,GENE,LAZY', 'OTLK,IOVA,ADVM,VIVE,LMB,ELTK,QIWI,BCDA,NAOV,MOXC,LMNL,LAZY,RAVE,VTVT,KOD,FANH,VTGN,ZYNE,WWR,MRTX'], ['OTLK,IOVA,ADVM,VIVE,LMB,ELTK,QIWI,BCDA,NAOV,MOXC,LMNL,LAZY,RAVE,VTVT,KOD,FANH,VTGN,ZYNE,WWR,MRTX', 'PROF,OTLK,DNJR,POAI,JAGX,TRNX,MYSZ,NOVN,TCON,BIMI,RAVE,BOSC,SALM,IRWD,VNET,CETXP,EVFM,ADVM,SINT,BELFA'], ['PROF,OTLK,DNJR,POAI,JAGX,TRNX,MYSZ,NOVN,TCON,BIMI,RAVE,BOSC,SALM,IRWD,VNET,CETXP,EVFM,ADVM,SINT,BELFA', 'BIOC,OTLK,OESX,RLMD,SESN,FCEL,TARA,JAKK,ASFI,RAVE,OPGN,UROV,AREC,KMPH,NOVN,KPTI,RGLS,ENG,IEA,CTIC'], ['BIOC,OTLK,OESX,RLMD,SESN,FCEL,TARA,JAKK,ASFI,RAVE,OPGN,UROV,AREC,KMPH,NOVN,KPTI,RGLS,ENG,IEA,CTIC', 'PAVMW,NAOV,WWR,DMPI,WKHS,RDVT,DFFN,CTRC,EQ,SMRT,KXIN,OSTK,ARWR,SESN,TBLTW,DGICB,CODA,AKCA,BSQR,USIO'], ['PAVMW,NAOV,WWR,DMPI,WKHS,RDVT,DFFN,CTRC,EQ,SMRT,KXIN,OSTK,ARWR,SESN,TBLTW,DGICB,CODA,AKCA,BSQR,USIO', 'OBLN,TARA,NOVN,WWR,ARTL,TNXP,AHPI,AGMH,DYAI,QRHC,JG,VTVT,VALU,KOD,TBLTW,VERI,MYT,QUMU,DMPI,KPTI'], ['OBLN,TARA,NOVN,WWR,ARTL,TNXP,AHPI,AGMH,DYAI,QRHC,JG,VTVT,VALU,KOD,TBLTW,VERI,MYT,QUMU,DMPI,KPTI', 'MYT,LMNL,TWMC,SANW,NMRD,SUNW,NTGN,BNSO,RAND,NVCN,GNUS,TVIX,EKSO,LTBR,OBAS,MYSZ,FSV,DNJR,BBQ,CPRT'], ['MYT,LMNL,TWMC,SANW,NMRD,SUNW,NTGN,BNSO,RAND,NVCN,GNUS,TVIX,EKSO,LTBR,OBAS,MYSZ,FSV,DNJR,BBQ,CPRT', 'PROF,OTLK,EPIX,CYCC,SUNW,IMUX,HIBB,PIRS,MVIS,OPRA,DZSI,CVCO,RLMD,NSEC,AGFS,USEG,ATEX,ELTK,NH,TLGT'], ['PROF,OTLK,EPIX,CYCC,SUNW,IMUX,HIBB,PIRS,MVIS,OPRA,DZSI,CVCO,RLMD,NSEC,AGFS,USEG,ATEX,ELTK,NH,TLGT', 'XELA,YIN,BCDA,USIO,SUNW,EDNT,AIHS,AVDL,THTX,SESN,VALU,ZN,CLGN,BIOC,PHIO,OPRA,WWR,SOLO,ARCT,SLGL'], ['XELA,YIN,BCDA,USIO,SUNW,EDNT,AIHS,AVDL,THTX,SESN,VALU,ZN,CLGN,BIOC,PHIO,OPRA,WWR,SOLO,ARCT,SLGL', 'PROF,ELTK,OBLN,RWLK,TTPH,TBLTW,OSN,CARA,CTHR,CPAH,TCCO,ZN,CMCO,OPNT,CY,VERB,APTO,AMRN,SGOC,BURG'], ['PROF,ELTK,OBLN,RWLK,TTPH,TBLTW,OSN,CARA,CTHR,CPAH,TCCO,ZN,CMCO,OPNT,CY,VERB,APTO,AMRN,SGOC,BURG', 'ONVO,OBLN,SSNT,MICT,EYEG,CAPR,TOCA,BLIN,ELTK,TITN,DOOO,NOVN,BCRX,VERI,PRVB,ICON,VERU,DYAI,NWGI,HIMX'], ['ONVO,OBLN,SSNT,MICT,EYEG,CAPR,TOCA,BLIN,ELTK,TITN,DOOO,NOVN,BCRX,VERI,PRVB,ICON,VERU,DYAI,NWGI,HIMX', 'DMPI,EDSA,AVEO,GNCA,OCUL,DGICB,RSSS,OPHC,CTHR,ONCT,OCGN,AIHS,SPNS,NTWK,HTBX,STRM,QTRX,TCCO,HJLI,MDRR'], ['DMPI,EDSA,AVEO,GNCA,OCUL,DGICB,RSSS,OPHC,CTHR,ONCT,OCGN,AIHS,SPNS,NTWK,HTBX,STRM,QTRX,TCCO,HJLI,MDRR', 'VISL,MRTX,ABIO,NK,CY,EYEG,ASLN,TLSA,GNCA,WKHS,MOGO,DYAI,ATXI,ATOM,SLS,ODT,MCEP,UMRX,NNDM,RESN'], ['VISL,MRTX,ABIO,NK,CY,EYEG,ASLN,TLSA,GNCA,WKHS,MOGO,DYAI,ATXI,ATOM,SLS,ODT,MCEP,UMRX,NNDM,RESN', 'WORX,ALNA,OESX,BURG,CWBR,VRML,NRBO,PETZ,APPS,BIOL,AYTU,BELFB,REFR,NH,TTD,SIEN,ZYNE,FOSL,ASNA,CETX'], ['WORX,ALNA,OESX,BURG,CWBR,VRML,NRBO,PETZ,APPS,BIOL,AYTU,BELFB,REFR,NH,TTD,SIEN,ZYNE,FOSL,ASNA,CETX', 'RWLK,NAKD,WKHS,HFFG,BCDA,YTEN,EOLS,RAVE,AMRH,EARS,ITRM,XTLB,CPAH,GNCA,IDN,ALNA,NTGN,WRTC,NSSC,CRON'], ['RWLK,NAKD,WKHS,HFFG,BCDA,YTEN,EOLS,RAVE,AMRH,EARS,ITRM,XTLB,CPAH,GNCA,IDN,ALNA,NTGN,WRTC,NSSC,CRON', 'LJPC,DTSS,AVDL,OESX,TBLTW,CMTL,TTNP,SFIX,OBLN,NOVN,PETZ,DYNT,CDTX,KBSF,TCCO,IDEX,PVBC,NAOV,CFRX,SNCR'], ['LJPC,DTSS,AVDL,OESX,TBLTW,CMTL,TTNP,SFIX,OBLN,NOVN,PETZ,DYNT,CDTX,KBSF,TCCO,IDEX,PVBC,NAOV,CFRX,SNCR', 'DXYN,ECOR,RWLK,TRMD,NEPT,LJPC,ENT,AVDL,OCUL,ARTL,BBQ,JCTCF,ZUMZ,AGMH,CRSP,MDB,AQMS,LWAY,CLSN,MHLD'], ['DXYN,ECOR,RWLK,TRMD,NEPT,LJPC,ENT,AVDL,OCUL,ARTL,BBQ,JCTCF,ZUMZ,AGMH,CRSP,MDB,AQMS,LWAY,CLSN,MHLD', 'PRVB,UONE,VUZI,TWMC,MHLD,WKHS,XBIO,ONVO,SHLO,CGIX,IDEX,CWBR,BLCM,AVCO,NAKD,AVDL,PTVCA,TEUM,RGLS,NURO'], ['PRVB,UONE,VUZI,TWMC,MHLD,WKHS,XBIO,ONVO,SHLO,CGIX,IDEX,CWBR,BLCM,AVCO,NAKD,AVDL,PTVCA,TEUM,RGLS,NURO', 'JAGX,HEBT,OXBR,VUZI,OCUL,CFRX,TRMD,OSMT,MKGI,ANY,DTSS,ZYNE,WWR,NRBO,NVFY,BPTH,TLGT,EYPT,HGSH,MTP'], ['JAGX,HEBT,OXBR,VUZI,OCUL,CFRX,TRMD,OSMT,MKGI,ANY,DTSS,ZYNE,WWR,NRBO,NVFY,BPTH,TLGT,EYPT,HGSH,MTP', 'OCGN,BCDA,BCOM,FCEL,CETX,SYBX,MVIS,PRTH,RKDA,AREC,PRTS,ENT,TOCA,BYSI,BIOL,SNSS,LOGC,IMRN,ENLV,TAIT'], ['OCGN,BCDA,BCOM,FCEL,CETX,SYBX,MVIS,PRTH,RKDA,AREC,PRTS,ENT,TOCA,BYSI,BIOL,SNSS,LOGC,IMRN,ENLV,TAIT', 'CODA,NDRA,SNSS,RRGB,ECOR,LMNL,STRM,JRJC,AEZS,SLRX,NEBUW,MRAM,OCUL,PFMT,DXLG,AXGT,OPGN,ENLV,NWGI,STIM'], ['CODA,NDRA,SNSS,RRGB,ECOR,LMNL,STRM,JRJC,AEZS,SLRX,NEBUW,MRAM,OCUL,PFMT,DXLG,AXGT,OPGN,ENLV,NWGI,STIM', 'OTLK,AVDL,CETX,MOXC,PETZ,HEBT,AFH,CIH,KMPH,CLSK,CLIR,HFFG,AXAS,VSTM,TC,AEZS,ITRM,JG,LWAY,MYOS'], ['OTLK,AVDL,CETX,MOXC,PETZ,HEBT,AFH,CIH,KMPH,CLSK,CLIR,HFFG,AXAS,VSTM,TC,AEZS,ITRM,JG,LWAY,MYOS', 'VISL,VSTM,ABUS,EIGI,SYBX,AGMH,AFH,DMAC,LONE,MOXC,MOTS,SYRS,AGFS,VIVE,SRNE,IOVA,JNCE,REKR,KERN,VERB'], ['VISL,VSTM,ABUS,EIGI,SYBX,AGMH,AFH,DMAC,LONE,MOXC,MOTS,SYRS,AGFS,VIVE,SRNE,IOVA,JNCE,REKR,KERN,VERB', 'MGI,CIFS,CNAT,GTHX,KERN,ONCT,SPWR,PETZ,SNDE,RGLS,LONE,TC,IPDN,MOXC,VSEC,IMRN,GLG,NCSM,OVID,CCRC'], ['MGI,CIFS,CNAT,GTHX,KERN,ONCT,SPWR,PETZ,SNDE,RGLS,LONE,TC,IPDN,MOXC,VSEC,IMRN,GLG,NCSM,OVID,CCRC', 'KERN,FRSX,AQB,ARCT,HEBT,MYOS,TLGT,DYNT,VBIV,VERB,MOXC,SVRA,SMRT,LYL,ENT,ENLV,TYHT,TLRY,AVRO,SPCB'], ['KERN,FRSX,AQB,ARCT,HEBT,MYOS,TLGT,DYNT,VBIV,VERB,MOXC,SVRA,SMRT,LYL,ENT,ENLV,TYHT,TLRY,AVRO,SPCB', 'PECK,EDSA,ARDS,BLU,ABUS,PAVMW,DMAC,LWAY,BCOM,HEBT,VBIV,CIFS,SYPR,SKYS,IDXG,SHIP,CGIX,FTR,ALJJ,TLGT'], ['PECK,EDSA,ARDS,BLU,ABUS,PAVMW,DMAC,LWAY,BCOM,HEBT,VBIV,CIFS,SYPR,SKYS,IDXG,SHIP,CGIX,FTR,ALJJ,TLGT', 'ARTL,TBLTW,BELFA,MBIO,LMNL,AXGT,WKHS,GEC,SNOA,ROSEU,OSTK,UEPS,EVLO,VSTM,BLRX,NCSM,WORX,OBSV,PT,MRKR'], ['ARTL,TBLTW,BELFA,MBIO,LMNL,AXGT,WKHS,GEC,SNOA,ROSEU,OSTK,UEPS,EVLO,VSTM,BLRX,NCSM,WORX,OBSV,PT,MRKR', 'VVPR,IMUX,KRYS,NERV,XBIO,VBIV,OBLN,BCDA,REKR,HUGE,USEG,ROSEU,HDSN,SGOC,CZR,HFFG,MARA,DTSS,TCCO,MCRB'], ['VVPR,IMUX,KRYS,NERV,XBIO,VBIV,OBLN,BCDA,REKR,HUGE,USEG,ROSEU,HDSN,SGOC,CZR,HFFG,MARA,DTSS,TCCO,MCRB', 'OXBR,NRBO,PAVMW,NDRA,RTTR,VERO,SYRS,VISL,OMEX,XNCR,ATOS,SSNT,LAZY,IPDN,MESO,REPL,CWBR,ABIO,ABUS,EDNT'], ['OXBR,NRBO,PAVMW,NDRA,RTTR,VERO,SYRS,VISL,OMEX,XNCR,ATOS,SSNT,LAZY,IPDN,MESO,REPL,CWBR,ABIO,ABUS,EDNT', 'XSPA,MARA,HFFG,GROW,OSTK,BCDA,VTSI,POAI,FRAN,INO,NXTD,MYGN,LLIT,PRTH,WTER,NMRD,RIOT,GNCA,USEG,ACST'], ['XSPA,MARA,HFFG,GROW,OSTK,BCDA,VTSI,POAI,FRAN,INO,NXTD,MYGN,LLIT,PRTH,WTER,NMRD,RIOT,GNCA,USEG,ACST', 'CETX,JAKK,ZGNX,FRSX,POAI,TEUM,TRCH,MLHR,IFMK,WTER,PAVMW,ATOS,FLL,FRTA,TGTX,AXSM,BCDA,QUMU,TBLT,VIRC'], ['CETX,JAKK,ZGNX,FRSX,POAI,TEUM,TRCH,MLHR,IFMK,WTER,PAVMW,ATOS,FLL,FRTA,TGTX,AXSM,BCDA,QUMU,TBLT,VIRC', 'CDMO,NBN,XBIO,GLG,PRPO,FRSX,HJLIW,CNST,ACRX,PHUN,VRCA,UMRX,CMRX,ONVO,GLMD,SRPT,SYRS,CUE,MPAA,CELC'], ['CDMO,NBN,XBIO,GLG,PRPO,FRSX,HJLIW,CNST,ACRX,PHUN,VRCA,UMRX,CMRX,ONVO,GLMD,SRPT,SYRS,CUE,MPAA,CELC', 'FCEL,IEA,TBLTW,OVID,PRPO,EMMS,AREC,AEMD,PPSI,NEON,NIU,PSTV,AVXL,SVRA,ACMR,XELB,PRNB,FRSX,CLGN,JAKK'], ['FCEL,IEA,TBLTW,OVID,PRPO,EMMS,AREC,AEMD,PPSI,NEON,NIU,PSTV,AVXL,SVRA,ACMR,XELB,PRNB,FRSX,CLGN,JAKK', 'FCEL,XELB,CRVS,RETO,DRTT,SLS,BRID,AMRN,KPTI,HJLIW,DGLY,AQMS,OSTK,PT,IMUX,ALYA,RBZ,DRRX,CYAD,IMRN'], ['FCEL,XELB,CRVS,RETO,DRTT,SLS,BRID,AMRN,KPTI,HJLIW,DGLY,AQMS,OSTK,PT,IMUX,ALYA,RBZ,DRRX,CYAD,IMRN', 'SLS,KPTI,FCEL,IPWR,TBLT,OSS,CRVS,SBPH,CYCCP,RETO,SSNT,CNAT,NAKD,TC,BIMI,AQB,CORV,RDIB,DVAX,EVLO'], ['SLS,KPTI,FCEL,IPWR,TBLT,OSS,CRVS,SBPH,CYCCP,RETO,SSNT,CNAT,NAKD,TC,BIMI,AQB,CORV,RDIB,DVAX,EVLO', 'TBLTW,TRNX,XBIO,TYME,BIOL,PIXY,NBSE,SLS,APEX,TBLT,NMCI,QTT,STKL,LPTX,MKGI,FCEL,TTPH,CBIO,BLCM,RETO'], ['TBLTW,TRNX,XBIO,TYME,BIOL,PIXY,NBSE,SLS,APEX,TBLT,NMCI,QTT,STKL,LPTX,MKGI,FCEL,TTPH,CBIO,BLCM,RETO', 'NAKD,PAVMW,TRNX,ADVM,NH,IDEX,EVOK,VIVE,BLFS,CETXP,TBLTW,ASLN,EDNT,OSTK,EIGR,SLGL,LTRX,LLIT,TC,ZN'], ['NAKD,PAVMW,TRNX,ADVM,NH,IDEX,EVOK,VIVE,BLFS,CETXP,TBLTW,ASLN,EDNT,OSTK,EIGR,SLGL,LTRX,LLIT,TC,ZN', 'TOPS,ACIA,SOLO,NEOS,ARTL,MOGO,EVK,XELB,USIO,MYOS,NVUS,NYMX,KNSA,PXS,EVOK,TACT,RWLK,UROV,RESN,GLBS'], ['TOPS,ACIA,SOLO,NEOS,ARTL,MOGO,EVK,XELB,USIO,MYOS,NVUS,NYMX,KNSA,PXS,EVOK,TACT,RWLK,UROV,RESN,GLBS', 'VISL,PAVMW,SEAC,FCEL,SGH,HLIT,PETZ,SIEB,AKTS,SNSS,TRIB,INPX,DTEA,NBSE,FPAY,JAKK,GWGH,SIEN,EPIX,NTEC'], ['VISL,PAVMW,SEAC,FCEL,SGH,HLIT,PETZ,SIEB,AKTS,SNSS,TRIB,INPX,DTEA,NBSE,FPAY,JAKK,GWGH,SIEN,EPIX,NTEC', 'CNAT,IMUX,CODX,FRG,GRNQ,XELB,NEPH,SLRX,RPAY,HEBT,ASNA,BMRA,HJLIW,ONVO,REKR,TWMC,NMCI,DMAC,CETX,AXGT'], ['CNAT,IMUX,CODX,FRG,GRNQ,XELB,NEPH,SLRX,RPAY,HEBT,ASNA,BMRA,HJLIW,ONVO,REKR,TWMC,NMCI,DMAC,CETX,AXGT', 'SGBX,NBSE,SES,MARK,IMV,INOD,SQBG,PECK,TRNX,MDGS,HJLIW,XBIO,UXIN,SRRA,MNOV,PRQR,GRNQ,FPAY,IFMK,EYEN'], ['SGBX,NBSE,SES,MARK,IMV,INOD,SQBG,PECK,TRNX,MDGS,HJLIW,XBIO,UXIN,SRRA,MNOV,PRQR,GRNQ,FPAY,IFMK,EYEN', 'CAPR,TRNX,HJLIW,EVOK,SES,BCOM,CSSE,REKR,PSTV,GLPG,AMTX,CETXP,AVRO,ICON,PERI,PROF,CSBR,AQB,ENLV,RLMD'], ['CAPR,TRNX,HJLIW,EVOK,SES,BCOM,CSSE,REKR,PSTV,GLPG,AMTX,CETXP,AVRO,ICON,PERI,PROF,CSBR,AQB,ENLV,RLMD', 'IMRN,BPTH,EPIX,TRNX,CGIX,FAMI,MOXC,DLPN,TRPX,IDXG,VERO,AVRO,CCRC,SPAR,UMRX,NK,CRIS,LONE,PHUN,BRQS'], ['IMRN,BPTH,EPIX,TRNX,CGIX,FAMI,MOXC,DLPN,TRPX,IDXG,VERO,AVRO,CCRC,SPAR,UMRX,NK,CRIS,LONE,PHUN,BRQS', 'SCPH,REKR,TEDU,AVRO,SGEN,SNCA,SSNT,YTRA,ANIX,MTSI,EVLO,SINT,FRAN,MIND,GWRS,IMUX,MICT,JCS,OPRA,INFN'], ['SCPH,REKR,TEDU,AVRO,SGEN,SNCA,SSNT,YTRA,ANIX,MTSI,EVLO,SINT,FRAN,MIND,GWRS,IMUX,MICT,JCS,OPRA,INFN', 'TEDU,OTLK,PAVMW,CODX,HEBT,USEG,XELA,NEPT,ACST,TBLT,ALPN,EOLS,GEVO,LLIT,BLPH,ACMR,FPAY,TBK,ENTX,OVID'], ['TEDU,OTLK,PAVMW,CODX,HEBT,USEG,XELA,NEPT,ACST,TBLT,ALPN,EOLS,GEVO,LLIT,BLPH,ACMR,FPAY,TBK,ENTX,OVID', 'MTP,AKTX,ANY,FCEL,ENT,ARCT,RRGB,ALTM,AFH,AZRX,TWMC,BBQ,SSNT,TBLTW,TBLT,HBIO,GNTX,MRTN,SCON,DRRX'], ['MTP,AKTX,ANY,FCEL,ENT,ARCT,RRGB,ALTM,AFH,AZRX,TWMC,BBQ,SSNT,TBLTW,TBLT,HBIO,GNTX,MRTN,SCON,DRRX', 'TRNX,DRRX,REKR,OBLN,BRQS,HBP,ATOM,ACST,GLBS,TEDU,EVOK,ZYNE,EMCF,TORC,ADAP,EDSA,FPAY,IDEX,HFFG,CETXP'], ['TRNX,DRRX,REKR,OBLN,BRQS,HBP,ATOM,ACST,GLBS,TEDU,EVOK,ZYNE,EMCF,TORC,ADAP,EDSA,FPAY,IDEX,HFFG,CETXP', 'TRPX,MBOT,OBLN,CHMA,ECOR,SLRX,TRCH,TBLT,PAVMW,PETZ,BHTG,KXIN,NBSE,CLIR,SES,NLTX,MKGI,INFN,STOK,GWGH'], ['TRPX,MBOT,OBLN,CHMA,ECOR,SLRX,TRCH,TBLT,PAVMW,PETZ,BHTG,KXIN,NBSE,CLIR,SES,NLTX,MKGI,INFN,STOK,GWGH', 'PLC,SGMA,NNDM,DUOT,DMPI,TER,MANH,TBLTW,ACST,KELYB,HHT,IART,INOD,CFMS,CNAT,WKHS,MDWD,TSEM,WTRH,TLSA'], ['PLC,SGMA,NNDM,DUOT,DMPI,TER,MANH,TBLTW,ACST,KELYB,HHT,IART,INOD,CFMS,CNAT,WKHS,MDWD,TSEM,WTRH,TLSA', 'MYSZ,ANIK,NRBO,IMV,EDAP,MRNS,AGLE,NTGR,SGMA,LOB,FCEL,REKR,QUMU,NAKD,DMRC,VRNA,DRIO,VC,SNSS,UFPI'], ['MYSZ,ANIK,NRBO,IMV,EDAP,MRNS,AGLE,NTGR,SGMA,LOB,FCEL,REKR,QUMU,NAKD,DMRC,VRNA,DRIO,VC,SNSS,UFPI', 'SMSI,HBIO,SLDB,LIVX,EHTH,OBLN,VCNX,SNBR,REKR,ALPN,XELA,SGMA,IMRN,DGLY,SLS,SNSS,BOXL,FLEX,MAT,AVGR'], ['SMSI,HBIO,SLDB,LIVX,EHTH,OBLN,VCNX,SNBR,REKR,ALPN,XELA,SGMA,IMRN,DGLY,SLS,SNSS,BOXL,FLEX,MAT,AVGR', 'PETZ,BCDA,TBLTW,BNTC,APOP,REFR,DTSS,ACST,LLIT,DLPN,VRCA,MYL,SNGX,PIRS,BVSN,AUTL,NCNA,HQI,ALRN,TORC'], ['PETZ,BCDA,TBLTW,BNTC,APOP,REFR,DTSS,ACST,LLIT,DLPN,VRCA,MYL,SNGX,PIRS,BVSN,AUTL,NCNA,HQI,ALRN,TORC', 'EFOI,IRMD,HTBX,ANTE,SLS,WRTC,AMKR,MEDP,GPOR,JAKK,RIBT,ZBRA,SMSI,BCDA,IZEA,CYCC,JJSF,SND,PI,PTEN'], ['EFOI,IRMD,HTBX,ANTE,SLS,WRTC,AMKR,MEDP,GPOR,JAKK,RIBT,ZBRA,SMSI,BCDA,IZEA,CYCC,JJSF,SND,PI,PTEN', 'BCDA,ENPH,BGFV,BNTC,LSCC,NVIV,RIOT,XELB,WKHS,MRCY,USEG,TBLT,NBSE,ARAV,REKR,PETZ,MICT,VSEC,HURN,MSTR'], ['BCDA,ENPH,BGFV,BNTC,LSCC,NVIV,RIOT,XELB,WKHS,MRCY,USEG,TBLT,NBSE,ARAV,REKR,PETZ,MICT,VSEC,HURN,MSTR', 'MYGN,MHLD,VNDA,EGOV,SPWR,ALIM,STAA,UMRX,ACAD,FIVN,CRUS,SELB,CASA,TVIX,IZEA,ALRN,VTGN,EFOI,SYRS,INOV'], ['MYGN,MHLD,VNDA,EGOV,SPWR,ALIM,STAA,UMRX,ACAD,FIVN,CRUS,SELB,CASA,TVIX,IZEA,ALRN,VTGN,EFOI,SYRS,INOV', 'XCUR,BL,CNXN,CNFR,SVMK,CHRS,DGLY,DRRX,BYFC,REFR,NWL,CORT,STML,PEIX,LOCO,SLS,BLDR,HMSY,ACTG,LOGC'], ['XCUR,BL,CNXN,CNFR,SVMK,CHRS,DGLY,DRRX,BYFC,REFR,NWL,CORT,STML,PEIX,LOCO,SLS,BLDR,HMSY,ACTG,LOGC', 'ALLK,TVIX,KMPH,VIIX,AIHS,MICT,MDWD,DXYN,IMGN,TTOO,HUGE,OSN,QMCO,LXRX,SQQQ,LIVX,RESN,TC,CYCCP,GROW'], ['ALLK,TVIX,KMPH,VIIX,AIHS,MICT,MDWD,DXYN,IMGN,TTOO,HUGE,OSN,QMCO,LXRX,SQQQ,LIVX,RESN,TC,CYCCP,GROW', 'EVER,LONE,APM,GLRE,SLRX,ALLK,PODD,TTOO,LYL,IFMK,AEZS,STML,ITRI,CLSK,GNMK,ZAGG,BHTG,CLRO,VG,LQDA'], ['EVER,LONE,APM,GLRE,SLRX,ALLK,PODD,TTOO,LYL,IFMK,AEZS,STML,ITRI,CLSK,GNMK,ZAGG,BHTG,CLRO,VG,LQDA', 'WW,QMCO,PRTK,SEDG,AVGR,MTCH,NEOS,GH,IOTS,TXMD,ASMB,NAKD,CYAD,DTSS,MGEN,SEEL,SCPH,GRNQ,KALA,SELB'], ['WW,QMCO,PRTK,SEDG,AVGR,MTCH,NEOS,GH,IOTS,TXMD,ASMB,NAKD,CYAD,DTSS,MGEN,SEEL,SCPH,GRNQ,KALA,SELB', 'CETXP,AOSL,CLSD,STMP,PAVMW,NCSM,GTHX,AMRS,CLXT,LUNA,ROKU,CERC,CDLX,STRM,PTE,PHUN,APEN,TLND,AGEN,CHNR'], ['CETXP,AOSL,CLSD,STMP,PAVMW,NCSM,GTHX,AMRS,CLXT,LUNA,ROKU,CERC,CDLX,STRM,PTE,PHUN,APEN,TLND,AGEN,CHNR', 'RKDA,INWK,CUTR,TTOO,SIEN,PCTI,CHNR,PBYI,IFMK,CRVS,GNUS,LXRX,APPN,VXRT,OSTK,TXMD,FCEL,ATLC,FAMI,CYRX'], ['RKDA,INWK,CUTR,TTOO,SIEN,PCTI,CHNR,PBYI,IFMK,CRVS,GNUS,LXRX,APPN,VXRT,OSTK,TXMD,FCEL,ATLC,FAMI,CYRX', 'RKDA,APOP,TBLTW,TBLT,DXYN,TRNX,POLA,ICON,SUMR,QUIK,DTSS,MRAM,MGEN,ALRN,AMAG,GNUS,INSE,NWGI,FRAN,TVIX'], ['RKDA,APOP,TBLTW,TBLT,DXYN,TRNX,POLA,ICON,SUMR,QUIK,DTSS,MRAM,MGEN,ALRN,AMAG,GNUS,INSE,NWGI,FRAN,TVIX', 'DNJR,DCPH,TRNX,NTEC,MYT,MBIO,NEWA,GTHX,AKER,VRAY,FORTY,DXYN,QMCO,PECK,JD,FBIO,EYPT,OPTN,NTGN,AEZS'], ['DNJR,DCPH,TRNX,NTEC,MYT,MBIO,NEWA,GTHX,AKER,VRAY,FORTY,DXYN,QMCO,PECK,JD,FBIO,EYPT,OPTN,NTGN,AEZS', 'AKTX,IEA,NVAX,JAKK,XNET,TVIX,SLDB,PRPL,MVIS,RKDA,PLXP,VIIX,ATHE,SYRS,AEMD,FVE,GROW,APEX,RDCM,SYPR'], ['AKTX,IEA,NVAX,JAKK,XNET,TVIX,SLDB,PRPL,MVIS,RKDA,PLXP,VIIX,ATHE,SYRS,AEMD,FVE,GROW,APEX,RDCM,SYPR', 'KXIN,SPTN,SINT,ENTX,SGRP,IFMK,AUTO,MICT,OSTK,RKDA,TROV,SPI,IPDN,ATEX,SGLB,LIVX,CIH,HYRE,FCEL,JAKK'], ['KXIN,SPTN,SINT,ENTX,SGRP,IFMK,AUTO,MICT,OSTK,RKDA,TROV,SPI,IPDN,ATEX,SGLB,LIVX,CIH,HYRE,FCEL,JAKK', 'PSTV,LMNL,SLDB,ORGO,TROV,NWGI,ALPN,CMCT,IPDN,METC,ARCT,NVAX,SMCI,QRHC,TH,ONCY,ICON,IPWR,APEX,FVE'], ['PSTV,LMNL,SLDB,ORGO,TROV,NWGI,ALPN,CMCT,IPDN,METC,ARCT,NVAX,SMCI,QRHC,TH,ONCY,ICON,IPWR,APEX,FVE', 'MGEN,TROV,ROSE,HWCC,RETO,QIWI,EPIX,AMRS,VTGN,TOPS,OPHC,FTSV,BASI,SES,GNUS,GRIN,SINA,ALRN,ELTK,AFH'], ['MGEN,TROV,ROSE,HWCC,RETO,QIWI,EPIX,AMRS,VTGN,TOPS,OPHC,FTSV,BASI,SES,GNUS,GRIN,SINA,ALRN,ELTK,AFH', 'WORX,CETXP,CETX,TRMD,GMDA,GENE,FAMI,MTSL,FRSX,RADA,TBLTW,BNGO,VVUS,TC,WWR,SUMR,NWGI,SCOR,IPWR,PHAS'], ['WORX,CETXP,CETX,TRMD,GMDA,GENE,FAMI,MTSL,FRSX,RADA,TBLTW,BNGO,VVUS,TC,WWR,SUMR,NWGI,SCOR,IPWR,PHAS', 'HJLIW,WTRH,LEDS,TTOO,RKDA,RBZ,ANY,AXAS,SMRT,EVOL,SQBG,HUGE,PDD,ASRT,MBIO,WKHS,CLUB,HYRE,ALNA,CLGN'], ['HJLIW,WTRH,LEDS,TTOO,RKDA,RBZ,ANY,AXAS,SMRT,EVOL,SQBG,HUGE,PDD,ASRT,MBIO,WKHS,CLUB,HYRE,ALNA,CLGN', 'NETE,KIRK,CETX,SG,FAMI,TRIL,LMNL,AIHS,PGNX,SGBX,XNET,SLRX,HEBT,WIFI,ACRS,ARTL,CTRN,SGRP,TELL,DRRX'], ['NETE,KIRK,CETX,SG,FAMI,TRIL,LMNL,AIHS,PGNX,SGBX,XNET,SLRX,HEBT,WIFI,ACRS,ARTL,CTRN,SGRP,TELL,DRRX', 'FFHL,CIH,YVR,TVIX,KMPH,EPIX,OPTT,AEMD,CLIR,OSN,APEX,VIIX,SNES,AEYE,SLRX,MOGO,DLPN,RESN,PLL,CCCL'], ['FFHL,CIH,YVR,TVIX,KMPH,EPIX,OPTT,AEMD,CLIR,OSN,APEX,VIIX,SNES,AEYE,SLRX,MOGO,DLPN,RESN,PLL,CCCL', 'HDSN,JAN,RKDA,CREG,ARDS,PHUN,PSTV,SLRX,BLCM,LIQT,PDCE,AMEH,WTRH,FBIO,ROSE,DXLG,CFMS,METC,CNST,DYNT'], ['HDSN,JAN,RKDA,CREG,ARDS,PHUN,PSTV,SLRX,BLCM,LIQT,PDCE,AMEH,WTRH,FBIO,ROSE,DXLG,CFMS,METC,CNST,DYNT', 'TBLTW,PSTV,DGLY,KOSS,PRTA,SSNT,IPDN,EDNT,CLUB,TBLT,NTEC,KXIN,MARK,ESEA,CDTX,PT,AKTS,PZZA,SNOA,AVDL'], ['TBLTW,PSTV,DGLY,KOSS,PRTA,SSNT,IPDN,EDNT,CLUB,TBLT,NTEC,KXIN,MARK,ESEA,CDTX,PT,AKTS,PZZA,SNOA,AVDL', 'NTEC,PSTV,DXYN,WTER,WHLR,DGLY,OTLK,PTE,LYL,SSNT,TOPS,PHAS,NBRV,LMFA,SCON,APDN,BWEN,IMUX,DAKT,DCAR'], ['NTEC,PSTV,DXYN,WTER,WHLR,DGLY,OTLK,PTE,LYL,SSNT,TOPS,PHAS,NBRV,LMFA,SCON,APDN,BWEN,IMUX,DAKT,DCAR', 'LOGC,DXYN,CREX,VTSI,NTNX,KTOV,SCVL,PEIX,CDTX,BELFB,IMBI,DMAC,DOOO,TECD,AEZS,HUGE,RDVT,AGTC,JAN,MTRX'], ['LOGC,DXYN,CREX,VTSI,NTNX,KTOV,SCVL,PEIX,CDTX,BELFB,IMBI,DMAC,DOOO,TECD,AEZS,HUGE,RDVT,AGTC,JAN,MTRX', 'ELTK,TLGT,SEAC,OPGN,AMBA,INSG,CFRX,AMRH,OPTN,XCUR,NVIV,YIN,SANW,PRTK,OPHC,NDRA,CNAT,EOLS,EMCF,URGN'], ['ELTK,TLGT,SEAC,OPGN,AMBA,INSG,CFRX,AMRH,OPTN,XCUR,NVIV,YIN,SANW,PRTK,OPHC,NDRA,CNAT,EOLS,EMCF,URGN', 'ABEO,CDTX,NETE,HDSN,CONN,SNSS,SAVA,TLRY,TTNP,TC,FAMI,MARK,TLSA,SEAC,USLV,WINS,TTOO,ATLC,HEBT,BIOL'], ['ABEO,CDTX,NETE,HDSN,CONN,SNSS,SAVA,TLRY,TTNP,TC,FAMI,MARK,TLSA,SEAC,USLV,WINS,TTOO,ATLC,HEBT,BIOL', 'PAVMW,OPGN,MRNS,EDNT,CFRX,RETO,BASI,ONTX,VRML,CLSD,GGAL,HIMX,BANFP,SGBX,GEVO,TOPS,ECOR,IPDN,EYEG,OVID'], ['PAVMW,OPGN,MRNS,EDNT,CFRX,RETO,BASI,ONTX,VRML,CLSD,GGAL,HIMX,BANFP,SGBX,GEVO,TOPS,ECOR,IPDN,EYEG,OVID', 'EDSA,ASNA,LE,GIII,AUTL,TRIL,CIH,VRML,NOVN,DSLV,ECOR,SCWX,LQDA,TRCH,MIK,NTGN,LYL,TEUM,APEX,ROSE'], ['EDSA,ASNA,LE,GIII,AUTL,TRIL,CIH,VRML,NOVN,DSLV,ECOR,SCWX,LQDA,TRCH,MIK,NTGN,LYL,TEUM,APEX,ROSE', 'MBOT,MACK,SCON,DOCU,SNDE,BYSI,TNXP,AUTL,AGTC,AMRH,RGLS,ORGO,CYRN,CUI,VERB,HOVNP,NTGN,NBSE,FRAN,NWGI'], ['MBOT,MACK,SCON,DOCU,SNDE,BYSI,TNXP,AUTL,AGTC,AMRH,RGLS,ORGO,CYRN,CUI,VERB,HOVNP,NTGN,NBSE,FRAN,NWGI', 'ACAD,FCEL,CYOU,YI,AESE,DSKE,EYEN,NTEC,BGFV,TOPS,PRTK,TTPH,GPOR,NH,MIK,DOGZ,AMTB,OSTK,BCDA,DGLY'], ['ACAD,FCEL,CYOU,YI,AESE,DSKE,EYEN,NTEC,BGFV,TOPS,PRTK,TTPH,GPOR,NH,MIK,DOGZ,AMTB,OSTK,BCDA,DGLY', 'BRPAR,FRAN,YI,YRCW,ACOR,AKBA,ADAP,DSKE,SCOR,ASNA,AKRX,LXRX,MESO,HIMX,FTR,AREC,IMMU,SPPI,ZEUS,GRNQ'], ['BRPAR,FRAN,YI,YRCW,ACOR,AKBA,ADAP,DSKE,SCOR,ASNA,AKRX,LXRX,MESO,HIMX,FTR,AREC,IMMU,SPPI,ZEUS,GRNQ', 'TTOO,HJLIW,FRAN,MTSL,LXRX,YRCW,OBLN,BELFA,VBIV,NH,BTAI,INNT,LIVX,DMTK,ENT,TRNX,FTR,CNAT,OTLK,NVCN'], ['TTOO,HJLIW,FRAN,MTSL,LXRX,YRCW,OBLN,BELFA,VBIV,NH,BTAI,INNT,LIVX,DMTK,ENT,TRNX,FTR,CNAT,OTLK,NVCN', 'SINT,WRLS,YI,EFOI,JAGX,EYEG,LMNL,TTPH,HDSN,OSS,SQBG,OSTK,TWNKW,EVLO,TTOO,ALIM,ALRN,RBZ,PCYG,EYPT'], ['SINT,WRLS,YI,EFOI,JAGX,EYEG,LMNL,TTPH,HDSN,OSS,SQBG,OSTK,TWNKW,EVLO,TTOO,ALIM,ALRN,RBZ,PCYG,EYPT', 'WINS,FRAN,MTSL,NTEC,NURO,OPGN,GFN,TBLTW,AMTX,DRAD,DGICB,KIN,DMTK,IRMD,BELFA,DTEA,DCAR,AMRH,DMPI,TRCH'], ['WINS,FRAN,MTSL,NTEC,NURO,OPGN,GFN,TBLTW,AMTX,DRAD,DGICB,KIN,DMTK,IRMD,BELFA,DTEA,DCAR,AMRH,DMPI,TRCH', 'WINS,CDEV,OAS,XOG,SNDE,SAEX,PTEN,NVAX,CYCC,QUIK,KOD,LXRX,RCON,GPOR,CIH,RADA,USIO,PECK,WVE,ZN'], ['WINS,CDEV,OAS,XOG,SNDE,SAEX,PTEN,NVAX,CYCC,QUIK,KOD,LXRX,RCON,GPOR,CIH,RADA,USIO,PECK,WVE,ZN', 'ACRS,BNTC,BCDA,APDN,FTFT,GLG,AVDL,AESE,AIHS,CLBS,CAPR,HGSH,PECK,SPWR,CJJD,APOG,JAGX,CGIX,CTXR,DNJR'], ['ACRS,BNTC,BCDA,APDN,FTFT,GLG,AVDL,AESE,AIHS,CLBS,CAPR,HGSH,PECK,SPWR,CJJD,APOG,JAGX,CGIX,CTXR,DNJR', 'SPI,HBP,LXRX,TTNP,ADXS,GRNQ,ECOR,ARAV,BVSN,AVDL,ASMB,HEBT,PRPH,ETTX,DTEA,PECK,EDSA,XELB,LMFA,PESI'], ['SPI,HBP,LXRX,TTNP,ADXS,GRNQ,ECOR,ARAV,BVSN,AVDL,ASMB,HEBT,PRPH,ETTX,DTEA,PECK,EDSA,XELB,LMFA,PESI', 'CHNR,EDSA,WWR,ENT,NWGI,AMTX,ARAV,MHLD,GRVY,TRNX,LYL,CTIB,TTOO,FWP,ALIM,REPL,CBPO,RTTR,MRUS,WINS'], ['CHNR,EDSA,WWR,ENT,NWGI,AMTX,ARAV,MHLD,GRVY,TRNX,LYL,CTIB,TTOO,FWP,ALIM,REPL,CBPO,RTTR,MRUS,WINS', 'CETXP,RBZ,BNTC,REPL,OPHC,ORGO,BURG,DISCB,GTIM,CLBS,BGFV,MTP,DRRX,EPSN,GVP,AKRX,VCNX,SLDB,NEBUW,APEN'], ['CETXP,RBZ,BNTC,REPL,OPHC,ORGO,BURG,DISCB,GTIM,CLBS,BGFV,MTP,DRRX,EPSN,GVP,AKRX,VCNX,SLDB,NEBUW,APEN', 'USAU,PAVMW,TBLTW,AVDL,ALIM,PSMT,BVSN,LYL,BCRX,USLV,SRAX,NAKD,OXFD,MHLD,MOGO,PXS,ASLN,PRTK,OVID,MKGI'], ['USAU,PAVMW,TBLTW,AVDL,ALIM,PSMT,BVSN,LYL,BCRX,USLV,SRAX,NAKD,OXFD,MHLD,MOGO,PXS,ASLN,PRTK,OVID,MKGI', 'NXTD,OVID,WHLR,NBEV,FRAN,BRPAR,SVRA,CTIB,MYGN,ELTK,VCNX,EVK,ZEAL,ESEA,MHLD,NNDM,SUMR,DUOT,RAND,CHFS'], ['NXTD,OVID,WHLR,NBEV,FRAN,BRPAR,SVRA,CTIB,MYGN,ELTK,VCNX,EVK,ZEAL,ESEA,MHLD,NNDM,SUMR,DUOT,RAND,CHFS', 'PHAS,IRCP,CMTL,TXMD,DOGZ,OVID,LIVE,CTRC,DSLV,RMBL,PERI,IRIX,ADXS,EFOI,FRAN,CPAH,CCRC,FTEK,LXRX,BNGO'], ['PHAS,IRCP,CMTL,TXMD,DOGZ,OVID,LIVE,CTRC,DSLV,RMBL,PERI,IRIX,ADXS,EFOI,FRAN,CPAH,CCRC,FTEK,LXRX,BNGO', 'PRPO,AMRH,MRNS,AEHR,SAEX,POAI,WISA,NNDM,DLPN,SYPR,JAKK,XBIO,UXIN,CETXP,TGEN,SKYS,SLS,SNDE,AUTO,TWMC'], ['PRPO,AMRH,MRNS,AEHR,SAEX,POAI,WISA,NNDM,DLPN,SYPR,JAKK,XBIO,UXIN,CETXP,TGEN,SKYS,SLS,SNDE,AUTO,TWMC', 'BVSN,TTPH,TLSA,AGFS,ESEA,SMMT,GRNQ,FCEL,EDSA,TRCH,AUTO,GRVY,WPRT,CCRC,ARTL,NTEC,XAIR,RCON,TMSR,CAPR'], ['BVSN,TTPH,TLSA,AGFS,ESEA,SMMT,GRNQ,FCEL,EDSA,TRCH,AUTO,GRVY,WPRT,CCRC,ARTL,NTEC,XAIR,RCON,TMSR,CAPR', 'DUOT,ENOB,AIRT,RTTR,CAPR,CCRC,VRCA,SMMT,VXRT,MOGO,JAKK,FORD,LOGC,IMRN,NK,OTIC,AEHR,SGEN,CYAN,CPST'], ['DUOT,ENOB,AIRT,RTTR,CAPR,CCRC,VRCA,SMMT,VXRT,MOGO,JAKK,FORD,LOGC,IMRN,NK,OTIC,AEHR,SGEN,CYAN,CPST', 'NAKD,CGIX,PT,INPX,CFMS,BKYI,CNST,BCDA,HTGM,FAMI,GERN,TBLTW,RSSS,HYRE,LLIT,NNDM,SES,OPHC,ENG,NVIV'], ['NAKD,CGIX,PT,INPX,CFMS,BKYI,CNST,BCDA,HTGM,FAMI,GERN,TBLTW,RSSS,HYRE,LLIT,NNDM,SES,OPHC,ENG,NVIV', 'TSG,RESN,APTO,SLNO,SURF,TNXP,IMRN,SQBG,TVIX,NAOV,AIMT,NAKD,MTP,PGNX,NTGN,VBIV,ZYXI,CBIO,AXSM,EPIX'], ['TSG,RESN,APTO,SLNO,SURF,TNXP,IMRN,SQBG,TVIX,NAOV,AIMT,NAKD,MTP,PGNX,NTGN,VBIV,ZYXI,CBIO,AXSM,EPIX', 'PDSB,MEET,MBOT,GNUS,CBIO,RBZ,TOCA,SRNE,ADXS,IMV,FCEL,TSRI,OXBR,ASNA,AMTX,FRAN,CLSD,BLRX,BVSN,MRSN'], ['PDSB,MEET,MBOT,GNUS,CBIO,RBZ,TOCA,SRNE,ADXS,IMV,FCEL,TSRI,OXBR,ASNA,AMTX,FRAN,CLSD,BLRX,BVSN,MRSN', 'TTNP,ECOR,MVIS,ZSAN,XELA,BVSN,NTEC,SGBX,SRAX,EYPT,CXDC,ALNA,EQ,AFH,YRCW,IZEA,OSN,EYEN,ADMS,NURO'], ['TTNP,ECOR,MVIS,ZSAN,XELA,BVSN,NTEC,SGBX,SRAX,EYPT,CXDC,ALNA,EQ,AFH,YRCW,IZEA,OSN,EYEN,ADMS,NURO', 'RTTR,CGIX,AKCA,YVR,XBIO,FTR,SFET,JNCE,MTSL,ARTL,ADXS,STKS,HTGM,ASNA,WINS,AEMD,OSS,PT,BVSN,OSTK'], ['RTTR,CGIX,AKCA,YVR,XBIO,FTR,SFET,JNCE,MTSL,ARTL,ADXS,STKS,HTGM,ASNA,WINS,AEMD,OSS,PT,BVSN,OSTK', 'PPSI,ASUR,LWAY,CIDM,TVIX,OXBR,STRM,SQBG,ASTC,FVE,RSSS,FAT,CETXP,INPX,SKYS,VCNX,EVK,WHLM,APDN,RETO'], ['PPSI,ASUR,LWAY,CIDM,TVIX,OXBR,STRM,SQBG,ASTC,FVE,RSSS,FAT,CETXP,INPX,SKYS,VCNX,EVK,WHLM,APDN,RETO', 'OCGN,RTTR,NSEC,DRIO,FPRX,REDU,TMSR,CATB,TRMD,TSRI,ENLV,BRPAR,CARV,YTEN,GENE,ECOR,SCON,TOPS,VIVE,PRPO'], ['OCGN,RTTR,NSEC,DRIO,FPRX,REDU,TMSR,CATB,TRMD,TSRI,ENLV,BRPAR,CARV,YTEN,GENE,ECOR,SCON,TOPS,VIVE,PRPO', 'RARX,SCON,TBLTW,ONCS,HJLIW,CGIX,SMIT,BBBY,EVOL,YVR,PRPL,BIMI,RETA,FLMN,CRVS,CLBS,TGEN,ZKIN,FRAN,CCRC'], ['RARX,SCON,TBLTW,ONCS,HJLIW,CGIX,SMIT,BBBY,EVOL,YVR,PRPL,BIMI,RETA,FLMN,CRVS,CLBS,TGEN,ZKIN,FRAN,CCRC', 'SES,TTNP,ENTX,CTXR,EFOI,DGLY,BIMI,ACRS,NAKD,FAST,AYTU,ELOX,GSM,SINT,PRPL,AERI,CTRC,CREG,RETA,SLDB'], ['SES,TTNP,ENTX,CTXR,EFOI,DGLY,BIMI,ACRS,NAKD,FAST,AYTU,ELOX,GSM,SINT,PRPL,AERI,CTRC,CREG,RETA,SLDB', 'BIMI,SES,IPWR,SAEX,MYT,RKDA,TOPS,SHIP,DMAC,BIOL,WWR,ENOB,VNET,ADVM,RVNC,CPSS,DARE,HHT,ENTX,CIH'], ['BIMI,SES,IPWR,SAEX,MYT,RKDA,TOPS,SHIP,DMAC,BIOL,WWR,ENOB,VNET,ADVM,RVNC,CPSS,DARE,HHT,ENTX,CIH', 'DTSS,RLMD,PETZ,WWR,RETA,DNJR,GRNQ,GLG,ASNA,CREG,ABEO,LLIT,AZRX,VRML,CNST,ANTE,AUPH,ZEAL,TCCO,JAN'], ['DTSS,RLMD,PETZ,WWR,RETA,DNJR,GRNQ,GLG,ASNA,CREG,ABEO,LLIT,AZRX,VRML,CNST,ANTE,AUPH,ZEAL,TCCO,JAN', 'BNGO,TBLTW,VIVE,ASMB,SQBG,OCGN,NVUS,DMTK,AEMD,CLSK,ENOB,ARTL,IPWR,CHFS,LIFE,TLSA,XBIO,WORX,BKYI,MHLD'], ['BNGO,TBLTW,VIVE,ASMB,SQBG,OCGN,NVUS,DMTK,AEMD,CLSK,ENOB,ARTL,IPWR,CHFS,LIFE,TLSA,XBIO,WORX,BKYI,MHLD', 'IPDN,CBLI,AFH,PLL,SES,AZRX,BIOL,JAKK,ASMB,FTFT,ASRT,LLNW,FRAN,DMPI,CLVS,AKRX,LQDA,AEZS,NDRA,ICON'], ['IPDN,CBLI,AFH,PLL,SES,AZRX,BIOL,JAKK,ASMB,FTFT,ASRT,LLNW,FRAN,DMPI,CLVS,AKRX,LQDA,AEZS,NDRA,ICON', 'BNGO,IPWR,AVGR,CLIR,OCGN,VTGN,XSPA,MTSL,DTSS,AFH,CREX,EKSO,VRML,TLGT,GENE,HMHC,CETX,SINT,FAMI,IMV'], ['BNGO,IPWR,AVGR,CLIR,OCGN,VTGN,XSPA,MTSL,DTSS,AFH,CREX,EKSO,VRML,TLGT,GENE,HMHC,CETX,SINT,FAMI,IMV', 'RLMD,XBIO,PETS,BIMI,ONTX,NSEC,CNST,SGEN,ARPO,PRQR,STRM,KNSA,COMM,TRCH,FSCT,PKOH,TWMC,SFET,SCOR,REFR'], ['RLMD,XBIO,PETS,BIMI,ONTX,NSEC,CNST,SGEN,ARPO,PRQR,STRM,KNSA,COMM,TRCH,FSCT,PKOH,TWMC,SFET,SCOR,REFR', 'KGJI,TROV,BIIB,STKL,ATNX,STMP,MYT,CVV,BLPH,ACRS,DVAX,CHCI,ZAGG,VRML,MVIS,CDEV,EVOL,PDSB,WORX,ETON'], ['KGJI,TROV,BIIB,STKL,ATNX,STMP,MYT,CVV,BLPH,ACRS,DVAX,CHCI,ZAGG,VRML,MVIS,CDEV,EVOL,PDSB,WORX,ETON', 'TBLTW,DXYN,SUNW,ADMP,CETXP,PBBI,APOP,TEUM,BRPAR,VSTM,OSMT,NVUS,MICT,CLSD,AXGT,CETX,PHIO,PDLI,IPWR,NVIV'], ['TBLTW,DXYN,SUNW,ADMP,CETXP,PBBI,APOP,TEUM,BRPAR,VSTM,OSMT,NVUS,MICT,CLSD,AXGT,CETX,PHIO,PDLI,IPWR,NVIV', 'TROV,VCNX,HUGE,MYT,UFPI,MICT,TSLA,RIGL,LTBR,SESN,AEYE,TEUM,MKSI,APTO,LRCX,ENTX,CUI,WISA,PDSB,KGJI'], ['TROV,VCNX,HUGE,MYT,UFPI,MICT,TSLA,RIGL,LTBR,SESN,AEYE,TEUM,MKSI,APTO,LRCX,ENTX,CUI,WISA,PDSB,KGJI', 'XNET,RBZ,LLIT,ANIK,MARA,NCTY,BOOM,BJRI,JAN,DOGZ,CNET,MRUS,CGIX,ROCK,AMAG,TCON,SREV,EYEN,CHEK,CREX'], ['XNET,RBZ,LLIT,ANIK,MARA,NCTY,BOOM,BJRI,JAN,DOGZ,CNET,MRUS,CGIX,ROCK,AMAG,TCON,SREV,EYEN,CHEK,CREX', 'ISEE,CLVS,AVGR,HTBX,AQST,CHNR,SCOR,ONCY,EKSO,XNET,CRVS,TGTX,LIVX,SHIP,SAFM,DTSS,CLUB,SNSS,ANTE,DXLG'], ['ISEE,CLVS,AVGR,HTBX,AQST,CHNR,SCOR,ONCY,EKSO,XNET,CRVS,TGTX,LIVX,SHIP,SAFM,DTSS,CLUB,SNSS,ANTE,DXLG', 'PROF,WINS,AMKR,SES,ONCY,ISEE,BLPH,TXRH,CTIB,TLSA,HLIT,MRTX,MOXC,STOK,SCON,ACRS,CYAD,SHOO,VRNS,EIDX'], ['PROF,WINS,AMKR,SES,ONCY,ISEE,BLPH,TXRH,CTIB,TLSA,HLIT,MRTX,MOXC,STOK,SCON,ACRS,CYAD,SHOO,VRNS,EIDX', 'ISEE,BGFV,SNCA,ONCY,OSPN,MFSF,JAGX,SSKN,YVR,KMPH,CLSD,VSEC,TENB,CROX,CPHC,CNET,MAT,DUOT,BNGO,CLSK'], ['ISEE,BGFV,SNCA,ONCY,OSPN,MFSF,JAGX,SSKN,YVR,KMPH,CLSD,VSEC,TENB,CROX,CPHC,CNET,MAT,DUOT,BNGO,CLSK', 'AGRX,ELOX,ABUS,DFFN,JCS,PTE,INPX,DWSN,EYEG,ENTX,STAA,CNSL,CRUS,SREV,OLED,CRAI,RILY,ABMD,REPH,ADMP'], ['AGRX,ELOX,ABUS,DFFN,JCS,PTE,INPX,DWSN,EYEG,ENTX,STAA,CNSL,CRUS,SREV,OLED,CRAI,RILY,ABMD,REPH,ADMP', 'CHFS,BGNE,OIIM,BLCM,ONVO,LOCO,IMGN,TNXP,REXN,GRIN,HOVNP,POAI,XENT,ACOR,RESN,QRVO,XOG,AMOT,RUTH,MMSI'], ['CHFS,BGNE,OIIM,BLCM,ONVO,LOCO,IMGN,TNXP,REXN,GRIN,HOVNP,POAI,XENT,ACOR,RESN,QRVO,XOG,AMOT,RUTH,MMSI', 'ALIM,ASFI,WMGI,AGRX,ALRN,EDNT,TUSK,YRCW,TRIB,RSSS,KLXE,BLCM,PRTS,GPOR,BSET,CLDX,FPAY,ONCY,LANC,OVID'], ['ALIM,ASFI,WMGI,AGRX,ALRN,EDNT,TUSK,YRCW,TRIB,RSSS,KLXE,BLCM,PRTS,GPOR,BSET,CLDX,FPAY,ONCY,LANC,OVID', 'SLS,AGRX,CETXP,FRTA,EVER,PRTK,IMGN,GAIA,CHNR,SHEN,VKTX,EVBG,ACOR,RKDA,RETO,EPZM,SGRY,TLSA,SYKE,SGLB'], ['SLS,AGRX,CETXP,FRTA,EVER,PRTK,IMGN,GAIA,CHNR,SHEN,VKTX,EVBG,ACOR,RKDA,RETO,EPZM,SGRY,TLSA,SYKE,SGLB', 'FCEL,CNST,WIFI,ASLN,SCOR,ADMP,TEUM,AQST,TRUP,CLSK,STKL,PRVB,FTSV,HTBX,CPSI,ISIG,ODP,PAYS,PTI,KE'], ['FCEL,CNST,WIFI,ASLN,SCOR,ADMP,TEUM,AQST,TRUP,CLSK,STKL,PRVB,FTSV,HTBX,CPSI,ISIG,ODP,PAYS,PTI,KE', 'MYT,BCOM,COLL,PESI,WKHS,DXCM,COMM,CLSK,PLUS,QTRX,ANGI,CLVS,PRTS,EYE,TMDI,SYNA,CHRS,MVIS,ZAGG,IQ'], ['MYT,BCOM,COLL,PESI,WKHS,DXCM,COMM,CLSK,PLUS,QTRX,ANGI,CLVS,PRTS,EYE,TMDI,SYNA,CHRS,MVIS,ZAGG,IQ', 'MICT,CLVS,TRUE,SIEN,FARM,AAXN,ENG,QNST,ADMS,SGLB,BCDA,SES,PBYI,MSON,SGMS,ACRS,MYT,TAST,DMAC,PCTI'], ['MICT,CLVS,TRUE,SIEN,FARM,AAXN,ENG,QNST,ADMS,SGLB,BCDA,SES,PBYI,MSON,SGMS,ACRS,MYT,TAST,DMAC,PCTI', 'USAU,HJLIW,LLIT,PHIO,AXAS,VERO,CUI,IEA,TRNX,MOXC,AEHR,JNCE,AEYE,YVR,SCON,SPRT,VTNR,CLSK,CNFR,DWSN'], ['USAU,HJLIW,LLIT,PHIO,AXAS,VERO,CUI,IEA,TRNX,MOXC,AEHR,JNCE,AEYE,YVR,SCON,SPRT,VTNR,CLSK,CNFR,DWSN', 'APDN,BREW,AIHS,JAGX,TBLTW,BNGO,AMRN,HJLI,FCEL,DMAC,SNCR,TNXP,ACST,DSKE,SNES,URGN,APPS,FTSV,VCNX,MYT'], ['APDN,BREW,AIHS,JAGX,TBLTW,BNGO,AMRN,HJLI,FCEL,DMAC,SNCR,TNXP,ACST,DSKE,SNES,URGN,APPS,FTSV,VCNX,MYT', 'NSYS,FCEL,CDLX,SCOR,MYT,MRSN,CLVS,TRPX,SREV,CLBS,ACIU,GNUS,ELTK,VTSI,KZIA,BKYI,JNCE,ALIM,BLNK,APEI'], ['NSYS,FCEL,CDLX,SCOR,MYT,MRSN,CLVS,TRPX,SREV,CLBS,ACIU,GNUS,ELTK,VTSI,KZIA,BKYI,JNCE,ALIM,BLNK,APEI', 'ASTC,MVIS,CLSN,AZRX,HROW,ALYA,RMBL,ASLN,ALIM,OPRA,WKHS,XBIT,SBPH,AEYE,AXNX,EQ,BLCM,PLXP,HBIO,SGLB'], ['ASTC,MVIS,CLSN,AZRX,HROW,ALYA,RMBL,ASLN,ALIM,OPRA,WKHS,XBIT,SBPH,AEYE,AXNX,EQ,BLCM,PLXP,HBIO,SGLB', 'AIHS,KERN,ANY,TMDI,CAAS,LASR,VRML,NWGI,BTAI,INTG,ONTX,KOD,TTOO,ONVO,LLIT,SQBG,DGII,CTRC,ASTC,ORGS'], ['AIHS,KERN,ANY,TMDI,CAAS,LASR,VRML,NWGI,BTAI,INTG,ONTX,KOD,TTOO,ONVO,LLIT,SQBG,DGII,CTRC,ASTC,ORGS', 'DRIO,TBLTW,RAVE,EVOK,CYCC,AEYE,ADXS,WTRH,CPST,VERB,SVRA,VISL,AGEN,EIDX,VTGN,ZEAL,LMNL,CLSN,NWGI,ELTK'], ['DRIO,TBLTW,RAVE,EVOK,CYCC,AEYE,ADXS,WTRH,CPST,VERB,SVRA,VISL,AGEN,EIDX,VTGN,ZEAL,LMNL,CLSN,NWGI,ELTK', 'SAEX,GNPX,ATNX,AGRX,AVGR,IDEX,AEYE,HTBX,CYAD,BCRX,NURO,FUV,ONTX,CRSP,TBLTW,IFMK,MGTA,AREC,TGEN,SRRK'], ['SAEX,GNPX,ATNX,AGRX,AVGR,IDEX,AEYE,HTBX,CYAD,BCRX,NURO,FUV,ONTX,CRSP,TBLTW,IFMK,MGTA,AREC,TGEN,SRRK', 'CGIX,KZIA,TBLTW,DFFN,ARAV,SURF,CLRB,KNSA,NBSE,ZN,MGTA,PTI,FLNT,CTMX,ELOX,IRIX,TRIL,CIFS,ISEE,YTEN'], ['CGIX,KZIA,TBLTW,DFFN,ARAV,SURF,CLRB,KNSA,NBSE,ZN,MGTA,PTI,FLNT,CTMX,ELOX,IRIX,TRIL,CIFS,ISEE,YTEN', 'SNCA,BIMI,IMTE,MTEM,EAST,IPWR,CTIB,AMTD,MYT,RAIL,PTI,TYME,KMDA,CYCC,AIRT,ACOR,PHIO,KERN,MRSN,LIFE'], ['SNCA,BIMI,IMTE,MTEM,EAST,IPWR,CTIB,AMTD,MYT,RAIL,PTI,TYME,KMDA,CYCC,AIRT,ACOR,PHIO,KERN,MRSN,LIFE', 'ASLN,EYEG,NCTY,KERN,CBAT,DMPI,VSTM,OPGN,AKBA,AXGT,TOCA,PT,ADMP,IIIV,HYRE,CGIX,OSMT,TPCO,FTR,HIBB'], ['ASLN,EYEG,NCTY,KERN,CBAT,DMPI,VSTM,OPGN,AKBA,AXGT,TOCA,PT,ADMP,IIIV,HYRE,CGIX,OSMT,TPCO,FTR,HIBB', 'SRNE,JAN,DFFN,MARA,CBAT,IDEX,ARTL,NLTX,PECK,CBLI,ZYXI,BLNK,GLG,CRESY,PT,CLVS,KGJI,AQST,ARWR,AKBA'], ['SRNE,JAN,DFFN,MARA,CBAT,IDEX,ARTL,NLTX,PECK,CBLI,ZYXI,BLNK,GLG,CRESY,PT,CLVS,KGJI,AQST,ARWR,AKBA', 'CCXI,TBLTW,CBLI,TBLT,SGBX,BCDA,PTI,KERN,VXRT,CBAY,CTIB,IRCP,CCCL,NLTX,NH,PT,ARWR,IMV,NTNX,XENE'], ['CCXI,TBLTW,CBLI,TBLT,SGBX,BCDA,PTI,KERN,VXRT,CBAY,CTIB,IRCP,CCCL,NLTX,NH,PT,ARWR,IMV,NTNX,XENE', 'ASLN,GRNQ,BKYI,ONTX,WISA,ZSAN,OVID,SRRA,JRJC,AEMD,AREC,QFIN,ABEO,OCGN,ORMP,UMRX,EYEN,RETO,SESN,OBSV'], ['ASLN,GRNQ,BKYI,ONTX,WISA,ZSAN,OVID,SRRA,JRJC,AEMD,AREC,QFIN,ABEO,OCGN,ORMP,UMRX,EYEN,RETO,SESN,OBSV', 'ASLN,CGIX,PT,SORL,NVAX,EVK,ADAP,HSDT,GLG,CLVS,SVRA,ONTX,ASTC,AKER,ADXS,ORMP,XCUR,TMSR,SBPH,MTEM'], ['ASLN,CGIX,PT,SORL,NVAX,EVK,ADAP,HSDT,GLG,CLVS,SVRA,ONTX,ASTC,AKER,ADXS,ORMP,XCUR,TMSR,SBPH,MTEM', 'KOD,ONTX,SRRA,YTEN,ASLN,ORMP,GLMD,BIMI,SAEX,HX,DRIO,SCON,ADXS,KERN,CAAS,ALIM,JNCE,CLSD,CBAT,SRNE'], ['KOD,ONTX,SRRA,YTEN,ASLN,ORMP,GLMD,BIMI,SAEX,HX,DRIO,SCON,ADXS,KERN,CAAS,ALIM,JNCE,CLSD,CBAT,SRNE', 'OTLK,DRIO,CRVS,KERN,VRAY,CLXT,TBLTW,ACOR,NTEC,NVIV,ABUS,TYHT,CTIB,LE,DLPN,EQ,ITRM,TTNP,ONTX,WRTC'], ['OTLK,DRIO,CRVS,KERN,VRAY,CLXT,TBLTW,ACOR,NTEC,NVIV,ABUS,TYHT,CTIB,LE,DLPN,EQ,ITRM,TTNP,ONTX,WRTC', 'PTGX,TBLTW,KOD,ALLK,RLMD,AVXL,LJPC,NTRP,GLMD,TYHT,DRIO,TUSK,BPTH,CPST,MBIO,BKYI,LXRX,TMSR,CUI,ELOX'], ['PTGX,TBLTW,KOD,ALLK,RLMD,AVXL,LJPC,NTRP,GLMD,TYHT,DRIO,TUSK,BPTH,CPST,MBIO,BKYI,LXRX,TMSR,CUI,ELOX', 'AUPH,QUIK,PTI,SCWX,ITRM,BKYI,DLTH,DMTK,SNDX,HOVNP,AEMD,STAF,BNGO,LTBR,POWL,MRNS,PDCO,ACAD,AIHS,CREG'], ['AUPH,QUIK,PTI,SCWX,ITRM,BKYI,DLTH,DMTK,SNDX,HOVNP,AEMD,STAF,BNGO,LTBR,POWL,MRNS,PDCO,ACAD,AIHS,CREG', 'PAVMW,KLXE,WEBK,CGIX,SRRA,AAME,DMTK,SAEX,ADXS,ONTX,DOMO,SNDE,JOUT,EPZM,CERC,EZPW,ASNA,LINC,ZUMZ,BIMI'], ['PAVMW,KLXE,WEBK,CGIX,SRRA,AAME,DMTK,SAEX,ADXS,ONTX,DOMO,SNDE,JOUT,EPZM,CERC,EZPW,ASNA,LINC,ZUMZ,BIMI', 'FTSV,XBIT,NLTX,FATE,APTO,HEBT,TYHT,SEEL,TGTX,CYCC,DMTK,PDLI,ITRM,ALIM,SNSS,ISEE,REKR,AVDL,GTYH,DVAX'], ['FTSV,XBIT,NLTX,FATE,APTO,HEBT,TYHT,SEEL,TGTX,CYCC,DMTK,PDLI,ITRM,ALIM,SNSS,ISEE,REKR,AVDL,GTYH,DVAX', 'TCCO,VERO,VTGN,ELOX,NLTX,AVCO,PT,EDNT,ITRM,GRTS,QUIK,ALPN,EQ,SNDE,MBOT,CDMO,PRNB,KLXE,MOTS,HGSH'], ['TCCO,VERO,VTGN,ELOX,NLTX,AVCO,PT,EDNT,ITRM,GRTS,QUIK,ALPN,EQ,SNDE,MBOT,CDMO,PRNB,KLXE,MOTS,HGSH', 'CTIB,FWP,BLCM,MRNS,DFFN,NTGN,IMUX,ABEO,CTIC,CLSK,GENE,SRAX,VERB,PLAB,TRIL,TOCA,APTO,ACRX,TWMC,OLLI'], ['CTIB,FWP,BLCM,MRNS,DFFN,NTGN,IMUX,ABEO,CTIC,CLSK,GENE,SRAX,VERB,PLAB,TRIL,TOCA,APTO,ACRX,TWMC,OLLI', 'ROSEU,ASRT,ASMB,GSM,FCEL,CTIC,TRIL,TOPS,TNXP,TRNX,XELB,AYTU,ISSC,PHAS,FRG,CLXT,ALNA,NDRA,CIFS,SBPH'], ['ROSEU,ASRT,ASMB,GSM,FCEL,CTIC,TRIL,TOPS,TNXP,TRNX,XELB,AYTU,ISSC,PHAS,FRG,CLXT,ALNA,NDRA,CIFS,SBPH', 'FRG,CCCL,SRPT,ADIL,NK,ACHV,CFRX,CREG,VTGN,SCPH,CBAY,BVXV,ENTX,SINO,CDXC,ACST,TRIL,LONE,HGSH,CARV'], ['FRG,CCCL,SRPT,ADIL,NK,ACHV,CFRX,CREG,VTGN,SCPH,CBAY,BVXV,ENTX,SINO,CDXC,ACST,TRIL,LONE,HGSH,CARV', 'AXSM,VTGN,VERB,TAOP,TA,ONVO,CLSD,LJPC,ONCY,BTAI,CFRX,HUGE,ASRT,IKNX,LAZY,TLC,RLMD,ABEO,OCGN,EYEG'], ['AXSM,VTGN,VERB,TAOP,TA,ONVO,CLSD,LJPC,ONCY,BTAI,CFRX,HUGE,ASRT,IKNX,LAZY,TLC,RLMD,ABEO,OCGN,EYEG', 'INPX,NEBUW,ONCY,PEIX,REGI,VERO,PRPO,TA,AXAS,NK,ACST,CLNE,IMGN,IFRX,NH,SNDE,CLLS,LOVE,NLTX,PSTI'], ['INPX,NEBUW,ONCY,PEIX,REGI,VERO,PRPO,TA,AXAS,NK,ACST,CLNE,IMGN,IFRX,NH,SNDE,CLLS,LOVE,NLTX,PSTI', 'TSRI,REED,INPX,PHAS,TLGT,FLNT,TEDU,EAST,SEEL,DMAC,SYRS,NAKD,XBIO,BLIN,ATOS,APTO,MCHX,CRNT,ESEA,STRM'], ['TSRI,REED,INPX,PHAS,TLGT,FLNT,TEDU,EAST,SEEL,DMAC,SYRS,NAKD,XBIO,BLIN,ATOS,APTO,MCHX,CRNT,ESEA,STRM', 'CBIO,HSDT,ZN,SELB,PLXP,NDRA,PRTK,ATHE,INO,PHAS,ABEO,AQMS,AXAS,ITRM,NOVN,SHLO,MSBF,RMBL,MTEM,SYRS'], ['CBIO,HSDT,ZN,SELB,PLXP,NDRA,PRTK,ATHE,INO,PHAS,ABEO,AQMS,AXAS,ITRM,NOVN,SHLO,MSBF,RMBL,MTEM,SYRS', 'DCAR,CETXP,MRNS,AGTC,ABUS,STCN,PRTA,NVUS,TRIL,SAVA,TOCA,CTXR,PHUN,LTBR,DFFN,FLL,AZRX,ARTL,SYRS,FCEL'], ['DCAR,CETXP,MRNS,AGTC,ABUS,STCN,PRTA,NVUS,TRIL,SAVA,TOCA,CTXR,PHUN,LTBR,DFFN,FLL,AZRX,ARTL,SYRS,FCEL', 'ITCI,LMNL,JRJC,MBOT,SAVA,CETXP,HX,IMTE,PT,SLNO,CALA,IDRA,CERC,DRRX,VBIV,GRNQ,PEIX,TRIL,PESI,OBSV'], ['ITCI,LMNL,JRJC,MBOT,SAVA,CETXP,HX,IMTE,PT,SLNO,CALA,IDRA,CERC,DRRX,VBIV,GRNQ,PEIX,TRIL,PESI,OBSV', 'HSDT,MBOT,PTI,TMSR,SLNO,SAVA,CETXP,AGTC,ZFGN,SURF,GRNQ,CTXR,PRVB,KOD,CFRX,ENTX,LLIT,AYTU,SSNT,CTIC'], ['HSDT,MBOT,PTI,TMSR,SLNO,SAVA,CETXP,AGTC,ZFGN,SURF,GRNQ,CTXR,PRVB,KOD,CFRX,ENTX,LLIT,AYTU,SSNT,CTIC', 'PAVMW,ONCY,TTPH,VRNA,LQDA,ACOR,FCEL,KMPH,CTXR,INPX,CTIB,ASRT,ACST,MRSN,DMAC,PAVM,YTEN,PT,CORV,ALDX'], ['PAVMW,ONCY,TTPH,VRNA,LQDA,ACOR,FCEL,KMPH,CTXR,INPX,CTIB,ASRT,ACST,MRSN,DMAC,PAVM,YTEN,PT,CORV,ALDX', 'ITRM,PT,AQMS,PDSB,MIK,ONTX,TSRI,SAVA,ORGS,YI,HJLI,CFRX,CDTX,RGLS,TLSA,ADXS,USWS,SRAX,FPAY,TRPX'], ['ITRM,PT,AQMS,PDSB,MIK,ONTX,TSRI,SAVA,ORGS,YI,HJLI,CFRX,CDTX,RGLS,TLSA,ADXS,USWS,SRAX,FPAY,TRPX', 'SVRA,SLGL,NVIV,OCGN,ONCY,ERYP,FCEL,RDIB,EYEG,FTR,CYCC,ARDS,TBLTW,OTIC,SEED,QUIK,TARA,TTNP,KXIN,SRAX'], ['SVRA,SLGL,NVIV,OCGN,ONCY,ERYP,FCEL,RDIB,EYEG,FTR,CYCC,ARDS,TBLTW,OTIC,SEED,QUIK,TARA,TTNP,KXIN,SRAX', 'FCEL,VIVE,INPX,BTAI,TBLTW,NVCN,HOVNP,LINC,JAGX,ONCY,SRAX,SBPH,KXIN,EVSI,TRIL,TYME,UROV,SLGL,NRBO,BLCM']]

In [2]:
for line in list1: 
    input_text, target_text = line #unpack input/target pair
    # use <s> as SOS
    # use</s> as EOS
    target_text = '<s>,' + target_text
    input_texts.append(input_text)     #in the same for-loop, ensuring correspondence
    target_texts.append(target_text)
    for symbol in input_text.split(','):
        if symbol not in input_symbols:  
            input_symbols.add(symbol)  #all undividable symbols of input seq
    for symbol in target_text.split(','):
        if symbol not in target_symbols:
            target_symbols.add(symbol) #all undividable symbols of target seq

In [3]:
input_symbols = sorted(list(input_symbols))     #tokenize input symbols ascending
target_symbols = sorted(list(target_symbols))  #tokenize output symbols ascending
num_encoder_tokens = len(input_symbols)     #num of encoder token
num_decoder_tokens = len(target_symbols)   #num of decoder token
max_encoder_seq_length = max([len(txt.split(',')) for txt in input_texts])    #max_len of encoder's input seq
max_decoder_seq_length = max([len(txt.split(',')) for txt in target_texts])  #max_len of decoder's target seq

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 250
Number of unique input tokens: 1277
Number of unique output tokens: 1278
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [4]:
input_token_index = dict(
    [(symbol, i) for i, symbol in enumerate(input_symbols)])    #input token: symbol, idx pair tuple
target_token_index = dict(
    [(symbol, i) for i, symbol in enumerate(target_symbols)])   #target token: symbol, idx pair tuple


In [5]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')    
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')   
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')    # teacher forcing and thus the same as decoder's input seq shape


In [6]:
decoder_input_data.shape

(250, 21, 1278)

In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, symbol in enumerate(input_text.split(',')):
        encoder_input_data[i, t, input_token_index[symbol]] = 1.     
        #input data: one-hot encoding. | i, t, one-hot
        
    for t, symbol in enumerate(target_text.split(',')):
        # decoder's target seq is one-timestampe shifted from its input seq; (we're using teacher forcing)
        decoder_input_data[i, t, target_token_index[symbol]] = 1.
        if t > 0:
            # decoder's target seq doesn't contain <s>
            decoder_target_data[i, t - 1, target_token_index[symbol]] = 1.
    #decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.    
    #decoder_target_data[i, t:, target_token_index[' ']] = 1.         

In [8]:
# define input seq and process
encoder_inputs = Input(shape=(None, num_encoder_tokens))  #None for batch size
encoder = LSTM(latent_dim, return_state=True)  #latent_dim: units
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  #returning 3 parts: outputs; state_h; state_c; tensor flowing
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]   #concate state_h and state_c

#take encoder's output states as the initialization states of decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# returned sequences will be used in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)  #unpack outputs, state_h, state_c
decoder_dense = Dense(num_decoder_tokens, activation='softmax') #just grid search, not beam search
decoder_outputs = decoder_dense(decoder_outputs)

#definte a model, taking encoder inputs and decoder inputs, and get decoder outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#early stopping
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, mode='auto', restore_best_weights=True)

#compile and train
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=2,
          epochs=epochs,
          validation_split=0.2,
         callbacks=[callback])  

# save the model for later use
model.save('s2s-lstm.h5')

Train on 200 samples, validate on 50 samples
Epoch 1/100
200/200 [==============================] - 6s 32ms/sample - loss: 6.6960 - accuracy: 0.0064 - val_loss: 6.7735 - val_accuracy: 0.0067
Epoch 2/100
200/200 [==============================] - 2s 12ms/sample - loss: 6.4475 - accuracy: 0.0098 - val_loss: 6.8908 - val_accuracy: 0.0057
Epoch 3/100
200/200 [==============================] - 2s 12ms/sample - loss: 6.3686 - accuracy: 0.0112 - val_loss: 6.9916 - val_accuracy: 0.0067
Epoch 4/100
200/200 [==============================] - 2s 12ms/sample - loss: 6.3125 - accuracy: 0.0124 - val_loss: 6.9196 - val_accuracy: 0.0048
Epoch 5/100
200/200 [==============================] - 2s 12ms/sample - loss: 6.2766 - accuracy: 0.0105 - val_loss: 6.9486 - val_accuracy: 0.0038
Epoch 6/100
200/200 [==============================] - 2s 12ms/sample - loss: 6.2431 - accuracy: 0.0131 - val_loss: 7.1008 - val_accuracy: 0.0076
Epoch 7/100
200/200 [==============================] - 2s 12ms/sample - loss: 6

In [9]:
# inference(sampling).
# methods as follows
# 1) encoder input and get the initialization states of decoder
# 2) with the initialization state and <s>, use decoder to predict next symbol
# 3) repeat the procedure

# define the sampling model
encoder_model = Model(encoder_inputs, encoder_states)  #decoder

decoder_state_input_h = Input(shape=(latent_dim,))  #state_h
decoder_state_input_c = Input(shape=(latent_dim,))  #state_c
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]  

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)  #3parts: output seq; state_h; state_c of decoder
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs) #the result
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)  #encoder's two states+decoder input seq | decoders output seq + decoder two states

# map token to symbol
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items()) #key-value pairs where token is the key, symbol is the value


def decode_sequence(input_seq):
    # input_seq to states_value
    states_value = encoder_model.predict(input_seq)

    # the sequence with len 1 as the <s>
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # the first symbol is the <s>
    target_seq[0, 0, target_token_index['<s>']] = 1.

    # for one batch and repeat sampling
    # take batch size as 1
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char + ','
        

        # stopping criterion : either hit the max_len or the stopping symbol
        if (sampled_char == '</s>' or
           len(decoded_sentence.split(',')) > max_decoder_seq_length):
            stop_condition = True

        # update the target_seq
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # update the states in the loop
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(1):
    # select sequences and decoder them
    input_seq = encoder_input_data[seq_index: seq_index + 1] #remain tensor of dim 3 by slicing not accessing
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: PHUN,NBSE,OCGN,BOXL,KTOV,BTAI,WHLR,XFOR,AZRX,SLRX,IMTE,EDAP,ENG,IZEA,SGOC,CHMA,VRML,NVUS,USEG,APOP
Decoded sentence: PROF,PROF,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,TBLTW,RBZ,RBZ,SNSS,SNSS,SNSS,SNSS,SNSS,


In [10]:
len(encoder_model.predict(encoder_input_data[0].reshape(-1,20,1277))[0][0]) #the lstm units dovetail

128